# Political Views Segmentation

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import dask
import dask_mongo
import dask.dataframe as dd

from langcodes import Language

In [2]:
sns.set_theme(style='darkgrid')

## Reading dataset in

In [3]:
from IPython.display import display
from dask.diagnostics import ProgressBar

In [4]:
df = dd.read_parquet('../parquet/users-full.parquet', engine='pyarrow')

In [5]:
df.columns

Index(['id', 'name', 'username', 'location', 'description', 'verified',
       'followers', 'followings', 'listed', 'tweets_liked', 'tweets',
       'created_at'],
      dtype='object')

In [6]:
df.dtypes

id                            int64
name                         object
username                     object
location                     object
description                  object
verified                       bool
followers                     int64
followings                    int64
listed                        int64
tweets_liked                  int64
tweets                        int64
created_at      datetime64[ns, UTC]
dtype: object

In [7]:
df.head(20)

,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
0,1237027988287471618,Palestina Internacional Broadcast,pbi_es,Palestina,Palestina Internacional Broadcast es la primer...,False,8524,130,40,694,9383,2020-03-09 14:50:40+00:00
1,1407899290790473732,Section Chine,SectionChine,,"Infos sans préjugés sur la Chine, avec une dom...",False,2107,2137,3,8243,457,2021-06-24 06:06:32+00:00
2,6509832,News18,CNNnews18,India,Official Twitter account of CNN-News18. Lightn...,True,4741218,397,6979,202,1028336,2007-06-01 20:31:01+00:00
3,1570222426570608641,Amazing Metaverse,AmazingMeta,Los Angeles,We are Amazing Metaverse! We are venturing int...,False,90,489,0,669,1180,2022-09-15 01:26:48+00:00
4,216893584,Periódico La Visión,lavisionatl,Atlanta Georgia,Empieza tu día informado con Periódico La Visi...,False,3167,949,70,2134,55632,2010-11-18 00:38:14+00:00
5,62089976,Tomasz 🇺🇦✌🏻🇵🇱🤝🇪🇺 #FBPE,yunec,"Kraków, Polska",🤷🏻‍♂️,False,7031,2903,20,222518,138614,2009-08-01 18:35:04+00:00
6,209520717,Oliva Noticias,olivanoticias,México,Centro Radiofónico de Xalapa - Grupo Oliva Rad...,False,26621,390,159,412,263814,2010-10-29 10:39:49+00:00
7,916681683134447617,Mila playporn,MilaPlayporn,,,False,8837,19,10,24,52336,2017-10-07 15:08:39+00:00
8,859964369932767232,MarioBekes,MarioBekes,"Sydney, New South Wales",World Class Podcaster ‘Life: THE Battlefield’ ...,False,1268,5001,4,9844,3436,2017-05-04 02:54:17+00:00
9,1499763123603050497,Fuck Putin Bot - #StandWithUkraine 🇺🇦,FuckPutinBot,,"I'm a bot. Every minute of every day, I tell P...",False,278,14,4,5,572077,2022-03-04 15:06:25+00:00


## Number of unique users

In [8]:
%%time

df.shape[0].compute(), df.shape[1]  # all entries

CPU times: user 8.19 s, sys: 872 ms, total: 9.06 s
Wall time: 7.48 s


(5922934, 12)

## Distribution by followers

In [13]:
%%time

followers_distribution = df['followers'].value_counts().compute()

CPU times: user 1.98 s, sys: 223 ms, total: 2.2 s
Wall time: 1.68 s


In [14]:
followers_distribution

followers
0           181073
1           115997
2            88923
3            73437
4            64430
             ...  
64713            1
64716            1
64717            1
64719            1
55072093         1
Name: count, Length: 75026, dtype: int64

In [17]:
followers_distribution_idxs = np.argsort(followers_distribution.index.values)
x = followers_distribution.index.values[followers_distribution_idxs]
y = followers_distribution.values[followers_distribution_idxs]

In [18]:
fig, ax = plt.subplots(figsize=(16, 12))
ax.plot(x, y)

ax.set_title('Distribution of Twitter users by followers count')

# ax.set_xticks(np.arange(0, 105, 5), minor=False)
# ax.set_yticks(np.arange(0, 45000, 2500), minor=False)

# ax.set_xlim(-3)
# ax.set_ylim(-1500)

ax.set_xlabel('followers')
ax.set_ylabel('twitter users having this many users')

plt.show()


KeyboardInterrupt



## Distribution by tweets

In [15]:
%%time

tweet_count_distribution = df['tweets'].value_counts().compute()

CPU times: user 2.8 s, sys: 248 ms, total: 3.05 s
Wall time: 1.9 s


## Distribution by location

todo: how do we normalize location names?

read from somewhere list of countries, cities and try to fuzzily match them?

maybe there is a library, which does that

## TODO further user exploration and maybe move simple statistics to a separate notebook?

## Matchine politically-inclined words in profile descriptions

In [7]:
import re

In [8]:
def alternative_regex(words: list[str]) -> str:
    return '|'.join([re.escape(word) for word in words])

In [9]:
def description_search(df: dd.DataFrame, regex: str, flags=re.IGNORECASE) -> dd.DataFrame:
    return df[df['description'].str.contains(regex, regex=True, flags=re.IGNORECASE)]

In [10]:
# TODO search for research in the internet?
political_words = {
    'nationalist': [
        'nationalist',
        'nationalism',
        'right-wing',
#         'god',
#         'family'
    ],
    'leftist': [
        'communist',
        'communism',
        'socialist',
        'socialism',
        'marx',
        'soviet',
        'left-wing',
        # TODO lgbtq? ...
    ],
    'democrat': [
        'democrat',
        'biden',
        'obama',
    ],
    'republican': [
        'republican',
        'trump',
        'MAGA'
    ]
    # todo
}

### Nationalist

In [14]:
nationalist_regex = alternative_regex(political_words['nationalist'])

In [15]:
with pd.option_context('display.max_colwidth', None), ProgressBar(dt=2.0):
    display(description_search(df, nationalist_regex).compute())

[########################################] | 100% Completed | 22.70 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
379,140546963,Kunal Khole,kunal_khole,"Pune, India",#Nationalist #ProudHindu,False,214,3135,0,38809,4017,2010-05-05 19:46:32+00:00
433,1440119985578266625,CHIBI RA YOUTUBE,CHIBI_RA_YT,France,"Todo lo que no puedes ver! Lo que no puedes ver! globalization is over, nationalism is here. \n2023 🐙vs🐊 (7)\nBusiness Contact 📩 Chibi.Ra69@outlook.com",False,136,105,1,81037,8768,2021-09-21 01:08:24+00:00
443,1576565569897132032,Divesh Pandey,DiveshP76528303,world,Free citizen of free country freely expressing the views with the center nationalism. And purely intentions of fulfilling national interests,False,254,62,0,12,366,2022-10-02 13:32:07+00:00
882,2264613618,Dbjena,dbadjena,India,"Truly nationalist, Free voice.",False,46,48,1,845,13986,2013-12-27 16:43:29+00:00
1417,1579099479143112704,Pran Sagari,PranSagari,India,Sanatan Nationalist | Geopolitics | History,False,0,17,0,8,3,2022-10-09 13:20:59+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
8026,1594770940969132032,DesiHathoda,DesiHathoda,,HyperNationalist,False,4115,3548,1,24772,8809,2022-11-21 19:13:42+00:00
10751,1561164849782169600,Rusty Dusty,dustyrusty1776,"Mexico city, Mexico","Family, God, and country.\nProud Christian Nationalist.\nAmerica should always come first. Extreme Ultra Nuclear MAGA with F-15's",False,145,157,2,16942,6177,2022-08-21 01:34:50+00:00
11667,1021461460751269888,Priya Singh,ipriyasinghbjp,"Ambikapur,Chhattisgarh India",धर्मो रक्षति रक्षितः | भारत माता 🇮🇳की सेविका |BJP Worker | Followed By PM @narendramodi ji | Nationalist🇮🇳 | RTs are not endorsements,False,11201,198,3,105427,25274,2018-07-23 18:26:04+00:00
12137,1425696926649917442,Harsh Vasita,HarshVasita1,,"Nationalist।\nstay low,stay simple📍\nPolitical Science||\nBikes||Football Enthusiast||",False,12,127,0,650,56,2021-08-12 05:55:52+00:00


### Leftist

In [16]:
leftist_regex = alternative_regex(political_words['leftist'])

In [17]:
with pd.option_context('display.max_colwidth', None), ProgressBar(dt=2.0):
    display(description_search(df, leftist_regex).compute())

[########################################] | 100% Completed | 51.80 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
34,1384611618655506437,Johannes Stern,JSternWSWS,,Chefredakteur und Herausgeber der deutschsprachigen World Socialist Web Site https://t.co/eohpRpt0r1,False,553,79,7,2907,1414,2021-04-20 20:56:32+00:00
158,85950775,Galatea,GalaToxica,México Lindo,👍 NRDA 🌺✨Socialism is Dead 🧐 Bk Bots 🚮 NO Porn 🤮 #ExUIA #FollowBack UnfollowBack too 🤷🏻‍♀️ NO DM🔥 WAR is Stupid!,False,11639,11659,151,56449,174685,2009-10-29 00:14:48+00:00
182,352182910,Natasha Tihamirova 🇦🇹🇺🇦🇺🇲,NGTRiga,"New York, USA","Democrat. Born in Riga during the USSR/communism era. I'm Russian, living in the US. Putin appalls me. Pro-choice, LGBTQ & BLM ally. I'm in animal rescue. 🐾🌊",False,9364,8436,134,17256,61098,2011-08-10 07:55:30+00:00
195,631181929,Gensui-kaigun-taisho Togo Heihachiro (Togo-sama),GensuiTogo,United States,Astrophysicist with an interest in Asian culture & history. Anti-alt-right. Anti-Fascist. Anti-Communist. Anti-Libertarian. Real 1st Amendment fundamentalist.,False,341,524,0,56162,26055,2012-07-09 15:22:21+00:00
264,337372191,Massimiliano Turcati,Travagliato,"Brescia, Lombardia ( Italia )","Contro il neo-liberismo, combatto nel mondo per un capitalismo dal volto umano, con un'iniezione di sano socialismo. A morte i paradisi fiscali.",False,395,309,4,71052,99756,2011-07-17 22:49:59+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
13020,108693859,Damián Acosta,damianacpi,"Mérida, Yucatán","Politólogo, bibliófilo, cocinero, marxista, amante del café, el jazz e hincha del buen fútbol... ¡todo un Proletario!",False,189,1326,1,3090,4322,2010-01-26 19:29:21+00:00
13356,273024626,Zim Hernandez,zimhernandez,"Sao Paulo, Brasil","🇨🇴 Marxista Tropical 🌴✊🏽, padre de dos gatos 🐈 y recomiendo discos en Tiktok ¡Sancocho o muerte!",False,208,1081,4,23742,6743,2011-03-27 17:30:19+00:00
14285,26277130,Pure Blood Deplorable,ConservativeRed,"Texas, USA","Jab refused! Not part of the 2-party circus. Nothing hatched in the District of Communism DC is deficit-neutral. END VAXX Ovens, Jab Mandates, & 🇺🇦 War!",False,2121,3087,68,718,6648,2009-03-24 17:15:19+00:00
14459,2822849653,Nick Sunderbruch,The_Legend56,"Sebring, Fl","21 They/Them\nAdministrator of the Continental Youth Assembly, former VP. Literally a communist, also constantly horny.",False,48,164,1,82,388,2014-09-20 21:31:51+00:00


### Democrat

In [18]:
democrat_regex = alternative_regex(political_words['democrat'])

In [19]:
with pd.option_context('display.max_colwidth', None), ProgressBar(dt=2.0):
    display(description_search(df, democrat_regex).compute())

[########################################] | 100% Completed | 22.59 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
182,352182910,Natasha Tihamirova 🇦🇹🇺🇦🇺🇲,NGTRiga,"New York, USA","Democrat. Born in Riga during the USSR/communism era. I'm Russian, living in the US. Putin appalls me. Pro-choice, LGBTQ & BLM ally. I'm in animal rescue. 🐾🌊",False,9364,8436,134,17256,61098,2011-08-10 07:55:30+00:00
201,104980346,Jack Young,JackGYoung,San Francisco Bay Area,Public Health | Researcher | Social & Racial Equality & Justice | Performing & Visual Arts | Literature | Travel | #BidenHarris #BLM #StopAsianHate #Warriors,False,2765,4809,17,39771,25815,2010-01-15 00:18:04+00:00
357,89609429,Political Sushi,politicalsushi,United States of America,Politics in the Raw - Working for Student Loan Relief #StudentDebt #StudentLoans #Biden2020,False,1086,994,12,46027,10009,2009-11-13 02:52:49+00:00
392,1235496401523654656,YosemiteJoe🇮🇷🇷🇺🇨🇳🇺🇸🇹🇩,joe_yosemite,,Primary account censored for posting #MichaelDAndrea’s elimination & suspended for posting #HunterBiden’s pics in seemingly pedophile acts,False,59,282,8,10084,6778,2020-03-05 09:24:46+00:00
449,1448298903216984066,🌻 Justice Now ⚖🌟🇺🇸,ChrisJustice01,,Biden/Harris 🌟🇺🇸🌊 \n#Resist ✊ #Resistance 🔥 \n#StandWithUkraine #StopPutin \n#JusticeIsComing #KeepTheFaith,False,9748,10709,38,226289,119002,2021-10-13 14:46:04+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
11847,1598854030759002118,Ali Luna,ALunaVenezuela,Guacara Carabobo,"Ingeniero Informatica / Docente UNITEC / Sec. Organizacion CAMBIEMOS Guacara. ""SOY DEMOCRATA""... Tu Me sigues, yo te sigo, asi podemos debatir y compartir ideas",False,89,314,0,1346,2926,2022-12-03 01:39:32+00:00
12593,1221520072541442048,Ashley Dermon Perry,DermonAshley,,"Our soul purpose is to help people. If you can't help them, at the very least,be kind to them. #BidenHarris 2020 #Wedidit (No DMs) Happily married!",False,3716,4986,5,26031,3275,2020-01-26 19:47:51+00:00
12654,1112848921,Simon Mulholland,ponyaccess,"Paisley, Scotland","Making Scottish countryside accessible to everyone. In every wheelchair, Everywhere. Obama the pony does the hard work.\nArchery in slammed chariot as hobby.",False,4439,4982,41,19656,25251,2013-01-22 22:52:20+00:00
13373,1518692316810260480,Shelly Bergman,ShellBell0530,"Ohio, USA",Mom of all boys. Follower of Christ. Republicans and Democrats are in it together. Wake up America!,False,987,2040,1,117,591,2022-04-25 20:44:14+00:00


### Republican

In [20]:
republican_regex = alternative_regex(political_words['republican'])

In [21]:
with pd.option_context('display.max_colwidth', None), ProgressBar(dt=2.0):
    display(description_search(df, republican_regex).compute())

[########################################] | 100% Completed | 22.57 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
36,1579458964382781440,MadHumanity,mad_humanity,,"MAGA is a fascist cult. // Religion is BS. // @elonmusk was BORN rich and is a dumb, egomaniacal baby. // Andy Taint is an insecure clown.",False,196,588,14,286,3136,2022-10-10 13:09:14+00:00
47,1221462414744596483,Republicans against Trumpism,RpsAgainstTrump,United States,Republicans conservatives fighting Trump & Trumpism. https://t.co/pWeVs4qkwx Please support our work: https://t.co/FkmisNic4X,False,398819,5086,1886,70355,79244,2020-01-26 15:58:45+00:00
132,1579157802680991745,Anna Larson 🇺🇦,AnnaLarsonLNC,,"Independent Journalist, Jewish✡️, exposing #MAGA for who they are, proudly supports #BLM✊🏿, #LGBTQ🏳️‍🌈, and most of all #Ukraine🇺🇦.",False,713,1252,1,93,382,2022-10-09 17:12:26+00:00
141,3044233482,H🎮CKEYSPAZ62🇺🇸,hockeyspaz62,USA,Censorship is un-American. American Patriot. MAGA. #WalkAway,False,604,907,8,162479,91942,2015-02-26 18:12:45+00:00
245,353074647,Fuzzy-Headed Realist #WhatAirdefenseDoing🇺🇸🇺🇦,Stivnik,DC,"Okinawan-Irish #NAFO #GuidedByVoices #TheFall Likes: Food, Beer, Dogs, Cats, Music. Hates: Trump & his enablers. I FOLLOW BACK UKRAINIANS & RESISTERS",False,848,1381,12,29932,12473,2011-08-11 14:23:59+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
14167,321740906,Dissident Aggressor to Christofascist Nationalists,AlienGurudeva,"Cydonia, Mars. The Red Planet.",One act of Kindness A Day. Vote. Never Trump! Keep Christofascism out of American Politics. 🏳️‍🌈 #LBGTQIA parent/grandparent. Just An Alien stuck on Earth꧁👽꧂,False,3863,4794,7,36809,38453,2011-06-22 02:03:51+00:00
14235,2757137027,EagleOne,AguiarJohn66,"Boston, MA",Love my Lord & Savior #Maga Republican \nIt's crucial we don't lose #2Amen #identity #history #Freedom\n\nBOSTON SPORTS RULE ✌️,False,2964,3830,2,7663,3703,2014-08-31 13:48:04+00:00
14242,189141459,sherrie philips,sherrieleep,southwest florida,"Love my country #maga #kaga hardworking taxpayer,love God, the farmers, nature+founding fathers,constitution, commonsense, mom, grandmom, sister, cousin, friend",False,1433,4334,1,61791,23674,2010-09-10 13:57:56+00:00
14283,1603914216594935809,SON DAKİKA,ain60478556,Turkey,"Güncel ,Ekonomi , Magazin , Spor , Dünya, Sağlık , Finans ,Teknoloji, Hava Durumu",False,1,3,0,0,0,2022-12-17 00:45:27+00:00


## Republican vs Democrats

In [73]:
democrat_userids = set()
republican_userids = set()

In [74]:
fdf = df  # filtered df

### 1st iteration (democrats)

In [75]:
regex = alternative_regex(['anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'])

In [77]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 30.92 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
2259,270921976,Michael O'Grady,mog7546,CANADA,"💚💚💚💚💚 Anti-Trump, Anti-Racist, Anti-homophobic...",False,51083,38529,203,366839,952463,2011-03-23 14:13:55+00:00
6208,1232270385514786816,70sChild,70sFella,,Journalist. Survivor of LGBT targeting. Polygl...,False,74,702,1,3736,8470,2020-02-25 11:45:41+00:00
8303,452663094,💋Political_Sass💋,leanne_cali,United States,💋Well Behaved Women Seldom Make History/ultra ...,False,3598,4513,6,166604,182451,2012-01-02 03:31:07+00:00
8947,65740000,The Real SB 🇺🇦✊🇺🇸,The_Real_SB,Yavin 4,Very former GOP. Xennial. Humanist. Geopolitic...,False,163,923,0,6382,2670,2009-08-14 21:24:36+00:00
9233,44080083,Tawfiq Karimi 🌻,tawfiqkarimi,"Bay Area, CA","Ajax, football (⚽️), Cycling, cricket and othe...",False,533,316,20,1350,16884,2009-06-02 07:33:56+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
6921,852697255945527296,LiberalDemocracyStan,CultureWarMyth,,Atheist |Anti-Trump and his cult of followers ...,False,376,1673,2,28054,20796,2017-04-14 01:37:22+00:00
11453,1469125789832691714,kathy 🐶🦊🐰🦁,kathysweetiecat,,lost old account…. democrat… all animals have ...,False,177,658,0,3633,40,2021-12-10 02:04:39+00:00
14646,28649563,Robert M Ramirez,robertr121,"Kerman, CA","Class of 1970, BLM, #TheResistance, dyed in th...",False,9753,10728,10,13436,30472,2009-04-03 20:49:32+00:00
671,64908654,Fox-Hawk,FOXtradamus,,"Anti-racist, anti-trump, anti-hate, anti-phobi...",False,67,275,0,2883,1758,2009-08-12 02:30:59+00:00


In [78]:
democrat_userids.update(sdf['id'])

In [79]:
for description in sdf['description'][:100]:
    print(description)
    print('-------------')

💚💚💚💚💚 Anti-Trump, Anti-Racist, Anti-homophobic, Anti Alt-White, #BlueWave #BLM, RESIST 💚💚💚💚💚  #Ukraine 🎸 🎸 🎸 #CANADA
-------------
Journalist. Survivor of LGBT targeting. Polyglot. Humour. #70s #PopularScience #Misandry #LGBTQ #Anti-Trump #AnimalWelfare 🇵🇹 🇩🇪 🇮🇹 🇫🇷 🇸🇪 🇸🇮🇺🇦
-------------
💋Well Behaved Women Seldom Make History/ultra potty mouth #BansOffMyBody #14thAmendment FUCK TRUMP #IllegitimateSCOTUS #IndictTrump #RussianGOP #IndictTraitors
-------------
Very former GOP. Xennial. Humanist. Geopolitical junky. Dad-bod model. Anti-MAGA. Anti-Putin. Pro-Democracy. Fly Eagles Fly. 🦅
-------------
Ajax, football (⚽️), Cycling, cricket and other sports, politics (in this sequence) Afghan-Dutch-American (not in this sequence) obviously Fuck Trump & Putin
-------------
Truth is stranger than fiction. 
Fuck Trump. #FuckPutin. Punch Nazis. Finally, the LA motherfucking RAMS.
-------------
Love 80s Music..... DC Comics from the Silver Age and Bronze age... Legion of Superheroes and Superman Ro

### 2nd iteration (republicans)

In [80]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'

In [83]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [84]:
regex = alternative_regex([
    'anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
    'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left'
])

In [85]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 60.24 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
3718,1551712179333828608,TheCityInvestor,cityinvestoruk,"London, England","Anti-migration, Anti-woke, Anti-leftism",False,336,557,0,0,156,2022-07-25 23:33:21+00:00
7730,1543604898880692224,Conservative Ambiance 🇮🇷☪️🇦🇹,ConservAmbiance,Protecting 1A & 2A,#MentalHealth Sufferer - #SexualAbuse Survivor...,False,241,287,0,1125,3611,2022-07-03 14:37:52+00:00
12616,1518714444511158273,american121,american12110,,"Christian, U.S. Constitution, Second Amendment...",False,36,162,0,126,193,2022-04-25 22:12:09+00:00
12172,1490327743745216512,The WEFXIT network,CrazyCanuk4life,Planet Ape,"#WEFXIT, no more; sustainable development goal...",False,117,150,0,1375,1533,2022-02-06 14:14:07+00:00
10309,1538180049324621824,RamblinDan,dan_ramble,USA,Anti-Putin Anti-Democrat Conservative,False,4,1,0,0,811,2022-06-18 15:21:46+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
8759,1518727753197142016,Jonathan Powers,RealJonathanP,,"🇺🇲✝️🇺🇲 Christian, #UltraMAGA, conservative, ga...",False,348,931,1,703,376,2022-04-25 23:04:57+00:00
9347,1056983154,Tony,tonyscotto10,"Myrtle Beach, SC",Forza Juve ⚫⚪\nForza Italia 🇮🇹\nFino Alla Fine...,False,483,243,1,14731,18099,2013-01-03 05:54:19+00:00
9944,1147401419592470528,BroadStBully,PhillyPsychoFan,"Ardmore, PA","Philly 4/4: Eagles, Flyers, Phillies, Sixers. ...",False,75,280,1,2495,4347,2019-07-06 07:06:30+00:00
9905,404744590,Erimas,erimas_a,DMV,International love #ProudFBA #BlackIndependent...,False,178,1206,1,1945,922,2011-11-04 10:47:01+00:00


In [86]:
# republican_userids.update(sdf['id'])

In [87]:
for row in sdf.itertuples():
    print(row.id)
    print()
    print(row.description)
    print('======================')

1551712179333828608

Anti-migration, Anti-woke, Anti-leftism
1543604898880692224

#MentalHealth Sufferer - #SexualAbuse Survivor - #MAGA - #Conservative - #Muslim - #LegalImmigrant - #1A & #2A Supporter - Anti Democrat - #MenToo
1518714444511158273

Christian, U.S. Constitution, Second Amendment, Pro - Life, U.S. Navy Vet, Ham Radio, Anti-Communist, Anti-Marxist, Anti-Democratic Party,
1490327743745216512

#WEFXIT, no more; sustainable development goal, stop the WHO agreement, NATO, NORAD, UN, WEF and all of this anti-democratic bullshit
1538180049324621824

Anti-Putin Anti-Democrat Conservative
43739838

Rejoiners and any other anti-democratic anti British types - don't bother - you'll be blocked.
1502031612120240143

Constitutional Conservative/Zionist/Proud Jew🇮🇱/Proud Russian🇷🇺Z/Anti-Leftism/AntiWok/AntiGlobalism/Freedom Over Everything! Preserve JudeoChristian values
1497529260684238851

Democrat💚Party European 
UA переможе Slava Ukraini ✌️🫂🦝  
🐈👩‍👧🌻midwife
@Miraculousmastodon@rhe

Too many false positives, checked only around 50 descriptions, and got over 10 false positives :(

In [ ]:
# should I remove those above from searches?

### 3rd iteration (republicans)

In [ ]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')

In [88]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [89]:
regex = alternative_regex([
    'ultra maga', 'ultra-maga'
])

In [90]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 14.25 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
7575,713020940976832512,▪️ J.P ▪️,NYCryptoKing,"New York, NY",🇺🇸 Patriot • Unvaxxed • Unafraid • 2A • ULTRA ...,False,3259,4907,32,2425,2470,2016-03-24 15:13:34+00:00
6144,1428584132871872513,Patriotbluejay🇺🇸,Jayson71787528,America,America first!! FAFO. Ultra maga force👑🇺🇸 Fuk ...,False,3,29,0,861,170,2021-08-20 05:06:27+00:00
8881,25947313,Michelle Maxwell,MichelleMaxwell,"Richmond, VA","Ultra-MAGA,Proud Virginia Constitutional Conse...",False,45939,24207,88,124988,40661,2009-03-23 03:45:52+00:00
9642,28670565,Xaymont,Xaymont,,Christ Following Patriot\n\nUltra MAGA\n\nQAno...,False,1074,1305,4,8985,13387,2009-04-03 22:25:35+00:00
1750,910663260,Chukwuma,ChukwumaOdelugo,,Lawyer. @auwcl alum. LSU/@SouthernU_BR Sch/Pub...,False,1134,3939,39,22677,14368,2012-10-28 16:47:13+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
12992,921857138150051840,Joseph R,JosephR28119167,,"retired contractor, Ultra MAGA, NRA life, no d...",False,1998,2206,2,36142,85520,2017-10-21 21:54:03+00:00
12436,1636568067349139457,Ultra MAGA Boy🇺🇸🍑,UltraMAGAboi,United States,ULTRA MAGA Gay Digital Solider - TRUMP 2024! M...,False,2063,2897,1,527,391,2023-03-17 03:20:31+00:00
63,1518933386106216448,Texas Patriot,GloriaW75356668,The Great State of TEXAS❤️,"Born again Christ follower, Patriot, America F...",False,309,572,0,3400,6346,2022-04-26 12:43:40+00:00
10057,1591860098149847043,Shad Olson,ShadOlson1989,"Piedmont, South Dakota","Author. Emmy, Edward R. Murrow and Marconi awa...",False,309,717,0,2663,2892,2022-11-13 18:26:57+00:00


In [91]:
republican_userids.update(sdf['id'])

In [92]:
for row in list(sdf.itertuples())[:100]:
#     print(row.id)
#     print()
    print(row.description)
    print('======================')

🇺🇸 Patriot • Unvaxxed • Unafraid • 2A • ULTRA MAGA 🇺🇸 

“Three things cannot be long hidden: the sun, the moon, and the truth.” —Buddha
America first!! FAFO. Ultra maga force👑🇺🇸 Fuk the demoncrats
Ultra-MAGA,Proud Virginia Constitutional Conservative, Fighting Everyday to Save This Republic 4 my Grandson 💙 Trump2024, 1A,2A,FAFO,GodBlessAmerica🙏🇺🇸🙏
Christ Following Patriot

Ultra MAGA

QAnon Decoder

Conspiracy Theorist

Crypto Enthusiast
Lawyer. @auwcl alum. LSU/@SouthernU_BR Sch/Pub Policy. SU alum. IRS. USAO DC. #PD DC. Civil rights. Animal rights. RTs usually endorsements. 'Ultra MAGA'
🚛Driving-Ultra Maga Nuclear MAGA⚡️Deadhead. 🚫🦗🖕🏻WEF 🪓🛡🇺🇸
Libertarian For Trump, Unvaccinated, Ultra MAGA, Semi Fascist, Deception Detector , praxeologist
Christian✝️, TRUMP LOYALIST ULTRA MAGA FLA POLL WORKER Puerto Rican Mom, Grandma, PRAYER WARRIOR🙏 REMOVE THE EVIL SWAMP 🇺🇸🇺🇸🇺🇸
Christian mama, small business owner, ultra maga -Lover of life, frequent high five giver
PATRIOT 🍿Island Life! 🌴#Galveston

### 4th iteration (republicans)

In [61]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'

In [62]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [118]:
regex = alternative_regex([
    'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
])

In [119]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 14.20 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
8583,771462178880778240,Samsonite07,kleensamsonite,Costa Rica,🇮🇱🇺🇸🇨🇿 God Before!! Pro-TRUMP-POUTINE-Bolsonar...,False,8422,6834,78,50650,70247,2016-09-01 21:38:11+00:00
6156,860860003376345088,Pascal PARIS 🇨🇵-🇮🇹 Vita ce n'è,pascalparis7,"Provence-Alpes-Côte d'Azur, Fr",Chanteur - Artiste intermittent \nANTI-MACRON!...,False,1565,2884,3,28997,6785,2017-05-06 14:13:13+00:00
4947,204774762,dein_erfolgs_dr,ACCRAWFORDUSA,Syke,CEO and Entrepreneur 🛩\n\nLoves his Family and...,False,1618,4991,14,4666,3165,2010-10-19 12:56:44+00:00
1548,1159850016120459264,SemperFidelis17,FidelisSemper17,,🇺🇸 🇮🇹 Patriot. Free Thinker. Pro Life. Pro Tru...,False,979,952,0,3401,1081,2019-08-09 15:32:47+00:00
2745,1481904230243971078,Jean Luc Macron ♂️🇫🇷🇷🇺🇺🇸,LameSansNom,Le Fenua,"Troll pro Poutine, pro Trump \nJe conchie macr...",False,948,346,15,64,75822,2022-01-14 08:21:39+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
8837,1520398298418491397,Angela,angela_allander,"Pennsylvania, USA",live and let live!everybody wants to be heard!...,False,1182,1327,0,3191,13385,2022-04-30 13:43:07+00:00
9422,1434603502655922185,jeanene wippert,wippert01,,Pro Trump pro vaccine pro life,False,3814,4997,0,3620,2577,2021-09-05 19:45:18+00:00
9802,1351335035958415360,Marigold,marigold6368,"Florida, USA","Pro-Dino Docs🦕🦖🇺🇸is worth it, Pro-Military, Pr...",False,419,2464,1,24718,15003,2021-01-19 01:06:09+00:00
10654,1213779065808674816,Committeeman Stephen Poindexter MAGA!,1978Steviep,United States,2012-13 Idaho Precent Committeeman. I speak my...,False,97,214,1,6841,5809,2020-01-05 11:07:53+00:00


In [122]:
republican_userids.update(sdf['id'])

In [120]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

🇮🇱🇺🇸🇨🇿 God Before!! Pro-TRUMP-POUTINE-Bolsonaro-ISRAËL- Anti-UE  
Blocked by ... Sellin, Henneton,....Clemenceau,. Araud, Faiche,Corbe,'Braana ,Haim,...😂
-----------------
Chanteur - Artiste intermittent 
ANTI-MACRON!  Un jour la liberté...
PRO-TRUMP

#Free #Frexit #Italexit 
#Italia #France #Belgium 👊💯👊
-----------------
CEO and Entrepreneur 🛩

Loves his Family and still believes in the American Dream 🇺🇲 

MAGA ❤️ PRO Trump  

🇩🇪 Syke - Miami 🇺🇸
-----------------
🇺🇸 🇮🇹 Patriot. Free Thinker. Pro Life. Pro Trump. Truth & God ✝️. MAGA. I follow back Patriots. Together we win. #WWG1WGA
-----------------
Troll pro Poutine, pro Trump 
Je conchie macron, zelensky, UE, biden et la clique progressiste.
y aura t il encore un général De Gaulle pour sauver la France
-----------------
DAV US Navy, pro Trump  Chef Electronics, Special Ed teacher, Inventor interested in Government, Anti-UN, NWO, Liberals and stupid snow flakes & more. Go Trump.
-----------------
I'm the toxic old woman who has no c

Found one false positive, but still most of them are ok :)

> FULLY VACCINATED AND BOOSTED!!  If you are pro-Trump - I will block you.  
> I stand with Ukraine 🌻🇺🇦🌻🇺🇦!!


### 5th iteration (democrats)

In [61]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'

In [62]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [148]:
regex = alternative_regex([
    'pro-biden', 'pro biden', 'pro-obama', 'pro obama',
    'pro-harris', 'pro harris', 'pro-pelosi', 'pro pelosi', 'pro-blinken', 'pro blinken',
    'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
])

In [149]:
regex = regex + r'|pro-democrat\b|pro-democrats\b|pro democrat\b|pro democrats\b|pro democratic party'

In [150]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 14.16 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
12445,2540474364,Tatsu De. Mtz 🔥❄🇺🇸🇲🇽🇨🇦🇯🇵🇺🇦🇫🇷🇩🇪,TatsuDeMtz,"Dallas, TX",OfficialTatsu🔥❄🇺🇸🇲🇽🇨🇦🇯🇵 (He/him)\nI'm no Jesus...,False,430,583,2,152021,114260,2014-06-02 00:28:18+00:00
1543,1348695358440931331,Ethan A. Kirchner #BLM #NewDeal4Animation (he/...,kirchner_blm,,Creator of The Weather Princess Guardians. I’m...,False,1058,5000,11,116498,98466,2021-01-11 18:17:16+00:00
803,47271647,Marie Van Pelt-Grier ☮️ 🇺🇦,weepelt,New Mexico,Progressive ideals. Tax the https://t.co/GnUjQ...,False,558,878,3,109141,160377,2009-06-15 05:53:34+00:00
6731,1497777944663670787,Maczek’s Ghost☮️,Maczeks_Ghost2,,Left for Tribel Social & Mastodon. See u there...,False,2563,3665,8,66485,30050,2022-02-27 03:38:07+00:00
6946,817914929671995394,"Elizabeth Bennet, Psy.D., J.D.",YosefaGitel,"Maryland, USA",Trauma Psychologist and Former Attorney. Pro B...,False,1175,3584,4,103736,106827,2017-01-08 02:04:50+00:00
9570,34792508,💙💛 Halfabubbleoff 🇺🇸 🌊 🇺🇸,halfabubbleoff,"Terra, Sol, OrionArm, MilkyWay",I decided to lock up w/ Melon’s takeover. BLM ...,False,1300,2337,2,117880,17826,2009-04-24 00:59:48+00:00
12107,1408561446443798549,Jorge,_Jorge_BCN,"Barcelona, España",“Ama a tu patria y lucha por tu país”. 🇪🇸 🇺🇦 🇵...,False,3016,4110,4,1315,4516,2021-06-25 23:03:26+00:00
8961,3411972585,🇺🇸 “Prosecute the Trump Crime Family”🇺🇸,skully1950,,"“PRO-BIDEN” Retired, Democrat, Honesty, Integr...",False,2541,2111,3,18103,25676,2015-08-10 06:10:42+00:00
14044,558819654,N Mc Nor Cal 🌻,NMc_NorCal,,"Pro-democracy. Unity yes, but with accountabil...",False,526,3399,21,79324,34677,2012-04-20 15:48:03+00:00
6041,2847561603,Marmot,GrizRich,,Pro Biden,False,522,1354,2,53308,12566,2014-10-28 01:30:00+00:00


In [153]:
democrat_userids.update(sdf['id'])

In [152]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

OfficialTatsu🔥❄🇺🇸🇲🇽🇨🇦🇯🇵 (He/him)
I'm no Jesus go fvck yourself 🔫, Look up the word Apadravya, Atheist? Pro-Democrat Pro-Biden/Harris, Science.
-----------------
Creator of The Weather Princess Guardians. I’m an autistic gamer, artist, voice actor, pro-Democrat, pro-LGBTQ+, anti-racist, pro-Black, and feminist.
-----------------
Progressive ideals. Tax the https://t.co/GnUjQ61ieR for all. Less money on wars..more on education. Against Trumpism/GOP. Pro Biden
My views are my own.
-----------------
Left for Tribel Social & Mastodon. See u there! Dad, Husband, psychotherapist; Life-Time Democrat;Pro-Biden; Build Back Better; 🇺🇦;BLM;🏳️‍🌈& 🏳️‍⚧️ Ally;
-----------------
Trauma Psychologist and Former Attorney. Pro Biden, Democrats & Reasonable, Compassionate Politics. Waiting for Trump & his Enablers to Be Held Accountable.
-----------------
I decided to lock up w/ Melon’s takeover. BLM EU-❤️ Pro-Biden I try to FB. Trailer park kid, paperboy, work&college, now pro. Please ask B4 putting on li

### 6th iteration (democrats)

In [155]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
# 5th: 'pro-biden', 'pro biden', 'pro-obama', 'pro obama', 'pro-harris', 'pro harris', 'pro-pelosi', 
#      'pro pelosi', 'pro-blinken', 'pro blinken', 'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
#      'pro-democrats', 'pro-democrat', 'pro democrats', 'pro democrat'

In [156]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [163]:
regex = alternative_regex([
    'anti-gop', 'anti gop'
])

In [164]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 16.34 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
11769,1284601894069927936,Carolyn Wood,Carolyn84905526,"Sacramento area, California","Retired Oncology RN, Anti Fascist. Resister. N...",False,5032,5351,9,219213,229418,2020-07-18 21:32:45+00:00
11778,1345886705883217920,Simontemplar,Simontemplarz,,"Pro democracy, anti Trump and anti GOP.",False,159,770,5,30619,16005,2021-01-04 00:16:26+00:00
12228,331279044,Henry Parker,hparker48,,Science; USMC Veteran; Atheist; anti-GOP; anti...,False,2038,2659,1,24419,32599,2011-07-07 22:50:47+00:00
10197,144668938,"MOMZILLA@Kath2252💙ProChoice,BLM,LBGQT,Peace&Love",Kath2252,"Btw Sing Sing & West Point, NY","Wife, Momzilla, daughter, sister, aunt, great-...",False,3170,815,158,553225,764964,2010-05-17 00:30:43+00:00
5923,105196227,Chris Ho 💉💉💉💉💉😷🇺🇦🇺🇦🇺🇦,chrishophoto,"Portland, Oregon","Professional Photographer in Portland, Oregon....",False,133,569,5,2523,7891,2010-01-15 17:05:28+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
2328,223540484,Tom Purcell,TJPurc,"Morrisburg, Ontario, Canada",Retired Canadian Air Force Avionics Technician...,False,107,1123,1,1934,1908,2010-12-06 17:37:28+00:00
2395,711025658541907968,Trekkie Kristen,TrekkieKristen,"Earth, I think",#Trekkie 🖖🏻; #scifi nerd; #atheist; #prochoice...,False,2300,3775,15,124777,13255,2016-03-19 03:05:02+00:00
2052,262187944,null,CoMoJDub,,"Pro-democracy, anti-hypocrisy, anti-💩🤡, anti-GOP",False,965,3896,11,192454,33980,2011-03-07 15:03:20+00:00
9354,15515039,Greg Edward (he/him),PortugalLiving,,Being Pro-Democracy means you are Anti_Fascist...,False,588,443,22,318,893,2008-07-21 13:59:01+00:00


In [166]:
democrat_userids.update(sdf['id'])

In [165]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

Retired Oncology RN, Anti Fascist. Resister. No DMs. Anti GOP. No Lists. Stand with Ukraine.
-----------------
Pro democracy, anti Trump and anti GOP.
-----------------
Science; USMC Veteran; Atheist; anti-GOP; anti-Rump; people rule; voting is our currency. Still work to do! Pulling for 🇺🇦
-----------------
Wife, Momzilla, daughter, sister, aunt, great-aunt, niece, cousin, friend, retired Teacher's Assistant, tutor, LUPUS Warrior, Anti-GOP, lifelong Democrat,
-----------------
Professional Photographer in Portland, Oregon. Bicycle Racing Fanatic, Atheist, Liberal Twitter Ranter. Pro-BLM, anti-GOP.
-----------------
Pro Ukraine. Pro LGBTQ. Pro brown people.
Anti supremacy, colonialism, religion, and imperialism. Anti vatnik, and above all else, anti GOP.
-----------------
Supporter of free markets, democracy, human/animal rights. Interest(s) in biotechnology.

Anti-Putin, Anti-GOP, Anti-Pelosi, Anti-Corruption. Reluctant Democrat
-----------------
Retired academic. 
Proudly pro-Europea

### Checkpoint, just in case :)

In [11]:
from typing import Iterable

In [167]:
len(republican_userids), len(democrat_userids)

(1979, 1044)

In [169]:
!ls ..

auth.json	  parquet
importer.py	  screenlog.0
importer_flex.py  scripts
kaggle_data	  ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip
notebooks


In [12]:
def checkpoint(republican: Iterable[int], democrat: Iterable[int]):
    with open('../parquet/republican.txt', 'w', encoding='utf-8') as f:
        f.write('\n'.join(map(str, republican)) + '\n')

    with open('../parquet/democrat.txt', 'w', encoding='utf-8') as f:
        f.write('\n'.join(map(str, democrat)) + '\n')

In [13]:
def load_checkpoint() -> tuple[set[int], set[int]]:
    with open('../parquet/republican.txt', 'r', encoding='utf-8') as f:
        ids = [id_.strip() for id_ in f.read().strip('\n').split('\n')]
        republican = set(map(int, ids))
    
    with open('../parquet/democrat.txt', 'r', encoding='utf-8') as f:
        ids = [id_.strip() for id_ in f.read().strip('\n').split('\n')]
        democrat = set(map(int, ids))
    
    return republican, democrat

In [174]:
checkpoint(republican_userids, democrat_userids)

### 7th iteration (democrats)

In [155]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
# 5th: 'pro-biden', 'pro biden', 'pro-obama', 'pro obama', 'pro-harris', 'pro harris', 'pro-pelosi', 
#      'pro pelosi', 'pro-blinken', 'pro blinken', 'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
#      'pro-democrats', 'pro-democrat', 'pro democrats', 'pro democrat'
# 6th: 'anti-gop', 'anti gop'

In [156]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [180]:
regex = alternative_regex([
    'vote blue'
])

In [181]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 14.24 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
1225,1355247557773955073,Coalsmoke,Coalsmoke1,,"Vote Blue! GenZ! @ChuckPfarrer, @iGenPolitics_...",False,2340,3831,3,15254,19467,2021-01-29 20:13:12+00:00
1258,553240313,Amanda,NoMoreDramaMa,Planet Earth,"🌊💙🏳️‍🌈#BlueCrew https://t.co/hR7SN7pDsj Mom, ...",False,4665,4231,7,69840,30289,2012-04-14 04:26:27+00:00
6640,15201747,Michael J. Toro,GrayFoxDown,"Brooklyn, New York",I've Never Seen Such a World... Not Even in Br...,False,4573,2175,59,1593,22452,2008-06-22 23:25:41+00:00
10342,998321838857052163,Botocks Bollucks,GuntherGill,"wtpForever, Maryland",Black Lives Matter Dark Brandon Rises Women’s ...,False,6740,6707,3,145887,67681,2018-05-20 21:57:29+00:00
12629,1391622551839731712,ZECIVICS2🌻🍉,zecivics2,,"Politics, Policy, Economics & Science, Vote Bl...",False,562,4999,3,12670,13206,2021-05-10 05:14:26+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
2905,1595271411077156864,Wet Willy,WetWillly3232,,the great reset is great isn’t it !? 81 millio...,False,24,146,0,1881,3149,2022-11-23 04:22:10+00:00
3015,1608700884556431360,Suzanne,WitchyBitchGirl,,Grandparent raising granddaughter. Vote Dem. ...,False,62,290,0,26454,266,2022-12-30 05:46:29+00:00
3344,4220538749,MackAttack,bruce_macksoud,,"retired school administrator, dog lover, Democ...",False,2648,4999,5,171692,15384,2015-11-18 19:44:02+00:00
4984,408963967,Jody Lawson,jodyl15,"Wisconsin, USA","Business owner, mom, grandma, cat mom. Vote Bl...",False,4010,4844,2,16528,7156,2011-11-10 03:26:00+00:00


Jackpot!!!

In [183]:
democrat_userids.update(sdf['id'])

In [182]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

Vote Blue! GenZ! @ChuckPfarrer, @iGenPolitics_, @genzforchange, @POTUS, @SenSanders, @CinC_AFU, @MarkHertling, @SenatorTester 🇺🇸💪👊❤️🇺🇦
-----------------
🌊💙🏳️‍🌈#BlueCrew  https://t.co/hR7SN7pDsj Mom, Teacher. She/her.  Be kind, think locally, act globally. #SlavaUrainii🌻 I follow back. Vote Blue!💙
-----------------
I've Never Seen Such a World... Not Even in Brooklyn. Vote Blue!
-----------------
Black Lives Matter Dark Brandon Rises Women’s Rights #SSM⚧ Vote Blue Down Ballot‼️DM 👍 NO Lists
-----------------
Politics, Policy, Economics & Science, Vote Blue, Resist GOP, plus Ukraine & More Ukraine who fights for their Freedom, European & Western Values
-----------------
Vote BLUE! Local, City, State, Federal. Dems win when we VOTE. No matter how hard they make it: VOTE. #BLM “Call Me By Your Name” is my favorite film.
-----------------
Vote Blue. Jeep Chick. Adopt don’t shop. Vegetarian. I don't sugar coat shit, I'm not Willy Wonka.
-----------------
Worked at UN & UNESCO. Political junk

### 8th iteration (democrats)

In [155]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
# 5th: 'pro-biden', 'pro biden', 'pro-obama', 'pro obama', 'pro-harris', 'pro harris', 'pro-pelosi', 
#      'pro pelosi', 'pro-blinken', 'pro blinken', 'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
#      'pro-democrats', 'pro-democrat', 'pro democrats', 'pro democrat'
# 6th: 'anti-gop', 'anti gop'
# 7th: 'vote blue'

In [185]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [196]:
regex = alternative_regex([
    'voteblue', 'vote-blue'
])

In [197]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 14.22 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
210,1576721829443653632,Fella Fart Sniffer,Beeyatch0,The Divided States of America,#DefundPolice #DefundMilitary #DefundNazis #Ab...,False,8,77,0,1405,617,2022-10-02 23:52:47+00:00
275,823702142896762880,OmaResists! 🌊 🌊💔,SouthernOmaEng,"Virginia, USA","Mother, Grandmother and Software Engineer. #re...",False,1449,2284,1,69344,28278,2017-01-24 01:21:09+00:00
771,1223246422503501824,Mary L Graves 🇺🇸🏳️‍🌈🇺🇦🌊🌊🌊,IlluminatingABM,"Ashtabula, OH",NO DMs #metoo Wife of 28 years Mom - 'mimi' o...,False,4600,5068,2,6597,9030,2020-01-31 14:07:42+00:00
909,3191478638,Stan Spak,stanspak,,"retired, husband, father, grandfather, brasseu...",False,9387,7874,117,616522,682951,2015-05-10 23:16:55+00:00
930,49522978,BlackCatMuse🐈‍⬛🐈‍⬛🐈‍⬛🐈☕️☯️🌎♈️,Xenophon9,Blue Resister in a Red State,#Resist #wtpBLUE #VoteBLUE #Equality #Justice ...,False,4407,4406,2,76259,71365,2009-06-22 03:27:54+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
2112,916352803093835777,Carmen Jennings 💉💉 + 💉💉,JenningsCarmenM,"Oklahoma, USA",Animal lover 🐾 Proud Democrat #Resist #TheRe...,False,2890,4891,4,91696,41854,2017-10-06 17:21:48+00:00
3367,22271059,Lori☀️,lorisun,,Mom. Wife. Teacher. Beach lover. News junkie. ...,False,538,2023,10,123646,14161,2009-02-28 16:37:16+00:00
3660,107899504,Debriorn,debriorn,"Arizona, USA",#Resister#ProChoice #BLM#BidenHarris #HateTrum...,False,3536,4660,4,139695,11969,2010-01-24 04:05:08+00:00
7682,713734701190479872,MerZ,emmy2logan,"Texas, USA",#RESIST #VOTEBLUE TX-OR-AK-TX,False,1431,1928,2,147068,47686,2016-03-26 14:29:48+00:00


In [199]:
democrat_userids.update(sdf['id'])

In [198]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

#DefundPolice #DefundMilitary #DefundNazis #AbolishIRS #AbolishCIA #AbolishFBI #AbolishDHS #AbolishICE #DeletePaypal #AbolishNATO #NeverVoteBlueOrRed #ACAB
-----------------
Mother, Grandmother and Software Engineer. #resist #progressive #independent #VoteBlue #HelpingOthers #Empathy #Love #Kindness #WearAMask #SaveOurPlanet 🚫DMs
-----------------
NO DMs #metoo Wife of 28 years  Mom - 'mimi' of 6 - #voteblue-  #BLM 💉x4 - #LGBTQIA+ #Artist #PTSD #ADHD #GAD ❇️💗
-----------------
retired, husband, father, grandfather, brasseur de merde #GetVaccinated #WearAMask #GetBoosted #VoteBlue2022 #StandWithUkraine #StopPutin #NeverVoteConservative
-----------------
#Resist #wtpBLUE #VoteBLUE #Equality #Justice #Animals #Science #Education #Climate #LivingWage #HealthCare #ChildCare #AntiFascist Tao TaijiQigong HD
-----------------
Signup -- https://t.co/EPMQx5i2dw
#VoteBlueInRunoffs
#COVIDIsNotOver
I'm riding the Blue Bus! 💙
-----------------
Professor
Grandmother #LifelongDemocrat
#Avidgolfer(begi

### 9th iteration (??)

In [201]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
# 5th: 'pro-biden', 'pro biden', 'pro-obama', 'pro obama', 'pro-harris', 'pro harris', 'pro-pelosi', 
#      'pro pelosi', 'pro-blinken', 'pro blinken', 'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
#      'pro-democrats', 'pro-democrat', 'pro democrats', 'pro democrat'
# 6th: 'anti-gop', 'anti gop'
# 7th: 'vote blue'
# 8th: 'voteblue', 'vote-blue'

In [202]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [226]:
regex = r'#joebiden\b|#biden\b'

In [227]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 12.18 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
6848,1265487884,BLM Antifa,Durrell62,"Westlake, TX",#Biden #NATO #AntiFa #BLM #FBI #NAFO #Fellas #...,False,27607,19300,71,1467,27455,2013-03-13 21:22:46+00:00
6354,1436939293,Deva Staten 🌻💛💙💜,sarajaynestaten,"Saint Louis, MO",MO District Director for AntifaSoups 🙋🏻‍♀️#Bid...,False,4332,4413,5,36360,42410,2013-05-17 23:28:56+00:00
11552,113208276,Hewitt Newton,HewittNewton,,Citizen of Earth caring deeply about mankinds ...,False,5861,6391,18,44699,94945,2010-02-11 02:01:51+00:00
12452,1563186801983901696,John-nie,JohnAlp72543803,,#Panafricain #France #cfa #Russie #Burkina #ma...,False,40,194,0,1789,2391,2022-08-26 15:29:37+00:00
2630,613988694,Linda Kyambadde,globalcitizenln,,unaplogetic support #Obama #BobiWine #Biden #K...,False,4443,4840,96,211696,341162,2012-06-21 01:15:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
10016,800570576020484098,Michael Grenier,greniermichael2,FL USA,"Musician, Atheist,Genealogy #BLM #LGBTQ+ Safe ...",False,5053,5759,0,13254,3175,2016-11-21 05:24:34+00:00
9690,1702604292,🌊 Boston Nana 🌊,aquamwoman,"Massachusetts, USA","Democrat 🌊🇺🇲🗽🌎 Proud Mom of 3 men, Nana of 4,...",False,10472,11323,1,31814,25141,2013-08-26 18:36:29+00:00
5448,1491856193194766338,Joe,ForzaItalia123,,Coffee lover ☕️ Animal lover 🐶⚽️🏀🏈 🎮✈️🇵🇱🇺🇸🇮🇹#B...,False,388,611,0,2294,1992,2022-02-10 19:27:01+00:00
441,953439182990598144,Linda,Linda89072922,USA,#GOODTROUBLE#BIDEN#THETIMEISNOW,False,25,125,0,789,214,2018-01-17 01:29:50+00:00


In [228]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

#Biden #NATO #AntiFa #BLM #FBI #NAFO #Fellas #Ukraine #SeditionHunters #MAGATerrorist #RussianASSet #MeidasTouch #MSNBC #HuffPost #NPR #CBS #BBC #ABC #Reuters
-----------------
MO District Director for AntifaSoups 🙋🏻‍♀️#Biden #SLE #PTSD #BansOffOurBodies #FuckSedition #FFS 🤦🏻‍♀️ Please ask before DMing. 💙🌊 Fluent in profanity.
-----------------
Citizen of Earth caring deeply about mankinds problems. #WeThePeople #BLM #AbolishTheElectoralCollege #resister #Biden #ConvictTrump #Blue2024 #drcole
-----------------
#Panafricain #France #cfa #Russie #Burkina #mali #Poutine #Biden #Obama #Afrique
-----------------
unaplogetic support #Obama #BobiWine #Biden #KHive; personal goal is 2 promote socioecon justice; guide to ppl is how they treat vulnerable stranger #GoBuckeyes
-----------------
#SaveTheEarth #ClimateChange #ClimateAction 🇱🇷⚖🗽🌏 😷💉 #Democracy #Freedom 💙🟧#JoeBiden 🇺🇦
-----------------
ut tibi sic alteri. do unto others as if they are you. rising tide floats all boats #Biden Slava Ukr

Quite ok, but there are definitely a few neutral, and a few false positives, so I will omit this, and use it only if those we have will not be enough.

### 10th iteration (democrats)

In [155]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
# 5th: 'pro-biden', 'pro biden', 'pro-obama', 'pro obama', 'pro-harris', 'pro harris', 'pro-pelosi', 
#      'pro pelosi', 'pro-blinken', 'pro blinken', 'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
#      'pro-democrats', 'pro-democrat', 'pro democrats', 'pro democrat'
# 6th: 'anti-gop', 'anti gop'
# 7th: 'vote blue'
# 8th: 'voteblue', 'vote-blue'
# 9th: none filtered ('#joebiden', '#biden')

In [229]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [252]:
regex = r'💙💙.*democrat|democrat.*💙💙|💙💙.*vote|vote.*💙💙'

In [253]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 22.50 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
3039,808112592,Stephen S.,PapaSteve45,"West Virginia, USA",Happily married to my soul mate. Father of dau...,False,3045,3556,1,32031,40164,2012-09-07 03:43:58+00:00
10874,377565477,Merritt,LiberalEffects,"Salem, OR",🌈 🇺🇦✌️🏳️‍🌈 💙💙 #Resist PLEASE 🙏 #VOTE #SaveOurD...,False,10480,10229,170,19128,79714,2011-09-21 19:38:57+00:00
9733,1351954150733410304,Mari,maricel23356266,,mom 💙 vote because our democracy is hanging by...,False,3490,3318,1,36813,42410,2021-01-20 18:08:01+00:00
8161,29877533,Jack 🌊🌊🌊🌊 #FBResist 🇺🇸🇺🇸I Stand with Biden,BlackBag1,"Phoenix, AZ","Proud USAF Veteran, Arizona Democrat, love our...",False,8361,8202,3,217692,204437,2009-04-09 01:05:16+00:00
114,4864605627,Bjd42,barbisblue,,Blue looks good on me! 🌊🌊💙💙Blue Voter……\nBe on...,False,11405,12009,1,104108,17291,2016-01-30 18:41:41+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
10249,1351523228741754882,jean,jean69180719,South Florida,4th gen Floridian 😷 💉 💙💙💙 liberal mom/Grammy #...,False,91,100,0,8500,2732,2021-01-19 13:33:56+00:00
12178,132963925,Andale Telewa,Telewa98,"Nairobi, Kenya","Christian|Social Democrat|@Springboks, @KenyaS...",False,492,950,13,12505,15078,2010-04-14 17:02:32+00:00
2643,16070105,mikegagne,mikegagne,maui hawaii,"I have a right wing nut allergy, woodworker, S...",False,3342,4928,11,19538,8945,2008-08-31 20:10:44+00:00
13405,1299762448510189571,Angel1028 AKA Janice C,Angel102817,"Bayonne, NJ",blue democrat till I die.Actually joined in 20...,False,4509,4602,1,13801,7662,2020-08-29 17:35:01+00:00


In [255]:
democrat_userids.update(sdf['id'])

In [254]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

Happily married to my soul mate. Father of daughters ❤ Grandfather 
great-grandchild. Staunch 
Democrat 💙💙💙💯.
Retired 2016. Graphic Artist, Real Estate.
-----------------
🌈 🇺🇦✌️🏳️‍🌈 💙💙 #Resist PLEASE 🙏 #VOTE #SaveOurDemocracy #LockThemAllUp #ExpandSCOTUS #ProChoice #LGBTQ #EndTheFilibuster #BLM #WomensRights #PsychNurse
-----------------
mom 💙 vote because our democracy is hanging by 🧵💙💙🌻🌻 Ukraine supporter 💉4x vaxxed 😷 EVERY ELECTION IS IMPORTANT VOTE!!! supports 🏳️‍🌈🏳️‍⚧️⚧️ BLM 🟦
-----------------
Proud USAF Veteran, Arizona Democrat, love our flag, 🌊🌊🌊🌊🌊🌊; #BidenHarris; #Resist; #BlueWave; #DemCast; #DemCastAZ;#wtpBLUE; 🇺🇦💙💙 https://t.co/xNahqsdScH
-----------------
Blue looks good on me! 🌊🌊💙💙Blue Voter……
Be on the side of democracy!Team Biden LIFELONGDEMOCRAT🗽🌊 NO🚫 DM'S🗽🗽🌊
-----------------
Working with my husband to save OUR ❤️Democracy #Blue Crew🌊🌊Vote 💙💙Blue2022/2024-Support 🇺🇦-BLOCK DM, No Lists, Equality for all!!!BLOCK all BITCOIN
-----------------
#GoDemocratsGo💙💙💙💙💙
#Antifa

### 11th iteration (democrats)

In [256]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
# 5th: 'pro-biden', 'pro biden', 'pro-obama', 'pro obama', 'pro-harris', 'pro harris', 'pro-pelosi', 
#      'pro pelosi', 'pro-blinken', 'pro blinken', 'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
#      'pro-democrats', 'pro-democrat', 'pro democrats', 'pro democrat'
# 6th: 'anti-gop', 'anti gop'
# 7th: 'vote blue'
# 8th: 'voteblue', 'vote-blue'
# 9th: none filtered ('#joebiden', '#biden')
# 10th: 'vote ... 💙💙', '💙💙 ... vote', 'democrat ... 💙💙', '💙💙 ... democrat'

In [229]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [263]:
regex = alternative_regex([
    'proud democrat', 'prouddemocrat', 'proud-democrat',
    'lifelong democrat', 'lifelongdemocrat', 'lifelong-democrat'
])

In [264]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 33.17 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
1444,16353821,DavidMEdwards 🌊🌊🎬🎥🇺🇸🇨🇦🇺🇦,DavidMEdwards,"Los Angeles, CA","Media Producer/Director. Proud Democrat, Faith...",False,4097,4790,4,10207,7291,2008-09-18 21:42:42+00:00
5632,1228413540182028291,Blue Crew Member supporting Democracy,stephen_cuddy,"Napa, CA",The TRAITORS have never been easier to spot. A...,False,1992,1827,3,105760,120195,2020-02-14 20:20:27+00:00
6458,3239371244,Michele Moore 💙🌊🌊🌊 😷,justmmmoore,Here and there. CO native,"Lifelong democrat, woman 40+ years in corporat...",False,1041,1552,0,19977,15784,2015-06-08 01:54:02+00:00
8248,1364260260555464711,Sheryl Post,ProudDoggieMom,,"Proud Democrat, PRES. BIDEN, VP HARRIS, PRO-CH...",False,13100,13540,31,242442,139324,2021-02-23 17:07:09+00:00
8454,455778782,(((glykosymoritis))) 🌻,glykosymoritis,Dublin,For a democratic secular masterless & borderle...,False,974,232,51,56533,34213,2012-01-05 14:15:21+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
1436,1573749312701444097,VictoriaRae,VictoriRaeW,4th greatest world economy CA,Retired from Higher Education. Animal lover. P...,False,144,760,13,12481,21086,2022-09-24 19:02:07+00:00
2750,4901456687,Deborah Boyd,teachermom19681,,Retired elementary school teacher. I’m married...,False,236,235,1,9009,10645,2016-02-12 14:23:24+00:00
4006,55476613,Jeff,yogijeff,SoCal,Lifelong Democrat. Populism is a scourge,False,665,1670,4,133567,8260,2009-07-10 05:04:45+00:00
6140,1444801512757157890,Maddie,AdoptaBlackDog,United States,Mom. Grandma of three grandsons. Dog lover. Hu...,False,629,627,1,4004,2255,2021-10-03 23:08:36+00:00


In [266]:
democrat_userids.update(sdf['id'])

In [265]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

Media Producer/Director. Proud Democrat, Faith in humanity. No DM's, Please. #RightMatters #TruthMatters #BlueCrew
-----------------
The TRAITORS have never been easier to spot. A vote for any Republican is a vote to end Democracy.  I am a proud Democrat. #BlueCrew #SupportUkraine 🇺🇦
-----------------
Lifelong democrat, woman 40+ years in corporate America making less than men while outworking them. Equality won’t happen without diversity in power. NO DMs
-----------------
Proud Democrat, PRES. BIDEN, VP HARRIS, PRO-CHOICE, DOGS AND CATS ARE FAMILY, SAVE OUR PUBLIC LANDS & WILDLIFE. ANTIFA MEMBER, HATE ORANGE POS, GQP.
-----------------
For a democratic secular masterless & borderless world; for individual & social autonomy;for lifelong democratic civic education with democratic aim #glykoreads
-----------------
Passion~ Politics💙🌵Proud DEMOCRAT. Pro choice. Lifelong News junkie...Foodie🍴🍕Wine 🍷Cat owner🐾https://t.co/kmJA2ijKeG. All opinions are my own.
-----------------
#BidenHarris #

### 12th iteration (democrats)

In [268]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
# 5th: 'pro-biden', 'pro biden', 'pro-obama', 'pro obama', 'pro-harris', 'pro harris', 'pro-pelosi', 
#      'pro pelosi', 'pro-blinken', 'pro blinken', 'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
#      'pro-democrats', 'pro-democrat', 'pro democrats', 'pro democrat'
# 6th: 'anti-gop', 'anti gop'
# 7th: 'vote blue'
# 8th: 'voteblue', 'vote-blue'
# 9th: none filtered ('#joebiden', '#biden')
# 10th: 'vote ... 💙💙', '💙💙 ... vote', 'democrat ... 💙💙', '💙💙 ... democrat'
# 11th: 'proud democrat', 'lifelong democrat'

In [269]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [275]:
regex = alternative_regex([
    'blue wave', 'blue-wave', 'bluewave'
])

In [277]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 16.20 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
1634,35070393,Carla Hope No Check,CarlaHope,,Geekette #Resist #FBR #BlueWave #BeAGoodPerson...,False,7818,7770,4,149256,11233,2009-04-24 23:02:04+00:00
4383,825481227545214976,B Stewart,_wbstewart,United States,#weDidIt If you are butt hurt cause tRump Lost...,False,18209,18455,3,83608,60245,2017-01-28 23:10:36+00:00
4492,2837671687,Kris E Cisneros,kris_cis,"Elk Grove, CA",Proud Nana💕 Current events 🇺🇸 mostly political...,False,294,1019,1,53366,11737,2014-10-02 06:27:14+00:00
5137,824709442444734464,🇺🇦 Slava Ukraini🌻🇺🇦💛💙. 🟧🟦,sharkelady,"Washington, USA","Librarian, historian, genealogist. #BLM #livel...",False,1093,1196,2,61447,7565,2017-01-26 20:03:48+00:00
5204,1456238282803752968,🇺🇸 Warp CORE: Critical ⚠️🇺🇦,Critical_CORE,"USA, N. America, Earth, Sol","♓Father, Widower, Artist, Musician, Humanist✌ ...",False,1034,1619,10,17032,8515,2021-11-04 12:34:34+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
12980,622593840,lisa wyman,lisawym16,"Scranton, PA",Democrate Blue wave 💙 we must ban together to ...,False,2821,4153,2,36372,23293,2012-06-30 04:55:11+00:00
2625,886297672088522754,padres_slammers,ciero1959,"San Diego, CA",Lover of SD Padres Baseball. Friar Faithful. L...,False,174,465,2,50138,7811,2017-07-15 18:53:26+00:00
12244,822636182219747329,Debra Korbach,KorbachDebra,"Massachusetts, USA","Navy vet, NP, nature lover, environmentalist #...",False,1410,4991,1,285164,172854,2017-01-21 02:45:24+00:00
2936,3415701141,Patty Rice,PattyR1948,"Ohio, USA",#Resister #NO DMs #Mygrandkidsfuture #Ihatetru...,False,5969,6007,1,62212,25778,2015-08-11 17:29:18+00:00


In [279]:
democrat_userids.update(sdf['id'])

In [278]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

Geekette #Resist #FBR #BlueWave #BeAGoodPerson #ProudAlly 🏳️‍🌈🏳️‍⚧️ #GoldenRule #ProtectEarth 🌎
-----------------
#weDidIt If you are butt hurt cause tRump Lost boo fucking hoo. #FBR #45WasNeverMyPresident, #ConvictTrump, #BlueWave, #neveragain, no porn.
-----------------
Proud Nana💕 Current events 🇺🇸 mostly political. 🚫 DMs. Ally 🏳️‍🌈. Anti-Corruption. Pro-Democracy. #ProChoice #BlueWave2022 #IStandWithUkraine
-----------------
Librarian, historian, genealogist. #BLM #livelikethemountainisout #bluewave #resistance #PRU
-----------------
♓Father, Widower, Artist, Musician, Humanist✌ #FBR ~ #Resist ~ Ally of #BLM ✊🏾 & #LGBTQ+🌈👉Proud #NeverTrump-er!~ #BlueWave2022 🌊~💉x3 ~😷🆙~Name: Core
-----------------
I hate racism, injustice and intolerance! #TeamTrudeau #Resist #UncleJoe #BlueWave I follow back! Prone to colourful language!  I block idiots!
-----------------
“I’m not going to censor myself to comfort your ignorance” Jon Stewart. #Resistance America is nation of immigrants. Don't give 

### Checkpoint 2

In [281]:
checkpoint(republican_userids, democrat_userids)

### 13th iteration (republican)

In [268]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
# 5th: 'pro-biden', 'pro biden', 'pro-obama', 'pro obama', 'pro-harris', 'pro harris', 'pro-pelosi', 
#      'pro pelosi', 'pro-blinken', 'pro blinken', 'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
#      'pro-democrats', 'pro-democrat', 'pro democrats', 'pro democrat'
# 6th: 'anti-gop', 'anti gop'
# 7th: 'vote blue'
# 8th: 'voteblue', 'vote-blue'
# 9th: none filtered ('#joebiden', '#biden')
# 10th: 'vote ... 💙💙', '💙💙 ... vote', 'democrat ... 💙💙', '💙💙 ... democrat'
# 11th: 'proud democrat', 'lifelong democrat'
# 12th: 'blue wave', 'blue-wave', 'bluewave'

In [285]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [288]:
regex = alternative_regex([
    'ultramaga'
])

In [289]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 16.30 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
1386,1432758278392451077,MAGA 🇺🇸 Mozart 🇺🇸,TahoeMozart,California Sierra Nevada Mts,🇺🇸 #ULTRAMAGA #Patriot 🇺🇸 #Pureblood 🇺🇸 #Depl...,False,2023,3084,1,3041,2534,2021-08-31 17:33:06+00:00
8500,1396913202940846085,The Joker,JokerUSA_1,,"🇺🇲🃏If you give a man gasoline, don't cry when ...",False,8216,8016,6,1508,58212,2021-05-24 19:37:30+00:00
10383,793553296468369408,RedTango,desuforu,USA,Stock trader. 2nd twit account. Jack killed 20...,False,773,824,2,5494,5781,2016-11-01 20:40:24+00:00
720,1501585218364452866,Joe Wegz,JosephWeglarz2,Planet Earth,Body Soul & Spirit\nFollows Jesus Christ with ...,False,1117,1824,4,4941,4304,2022-03-09 15:46:57+00:00
8402,1596314776954511360,CleetusJenkins,FrredomEnforcer,"Kansas City, MO",#ProLife #NRA #RedWave #Trump2024 #TruePatriot...,False,25,60,0,71,54,2022-11-26 01:28:09+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
9626,48273142,MAGA Gangster ███,jeremyrooks,"████, OK","Dad of 2, Husband to @MyMaryKayBiz, Huge #okst...",False,322,826,11,31992,12993,2009-06-18 06:49:23+00:00
1453,4530195132,@gtl247,eytan1212,"Moved out of CA, now in AZ",IDF paratrooper🇮🇱 #MAGA 🇺🇸 support Israel 🇮🇱🙏🇺...,False,3645,3926,6,72714,78537,2015-12-11 17:07:36+00:00
12151,1482868474212864000,Patriot_Pep,Patriot_Pep,AZ,"*Account V*\nWhen America becomes 1984,\nThen ...",False,737,1672,0,6689,1667,2022-01-17 00:13:04+00:00
5879,20400202,Jeannie Valentine O’Neil,jvalentine613,"Hobe Sound, FL",Conservative election law attorney living in F...,False,1049,2514,17,54440,19552,2009-02-08 23:05:50+00:00


In [291]:
republican_userids.update(sdf['id'])

In [290]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

🇺🇸 #ULTRAMAGA #Patriot 🇺🇸 #Pureblood 🇺🇸  #Deplorable  🇺🇸 Avid Outdoorsman 🇺🇸  Retired, but busy AF 🇺🇸 1st 15K acct purged on 1/8/21 🇺🇸 GO #Trump!
-----------------
🇺🇲🃏If you give a man gasoline, don't cry when he burns the world 🃏🇺🇲 #UltraMAGA #1 #2A 
Support our Constitutional Republic, we must defend it 🇺🇲👊🏼
-----------------
Stock trader. 2nd twit account. Jack killed 2008 acc. Left: abort your babies, clean the gene pool. DMs useless. Right is Right. #Trump #DeSantis  #ULTRAMAGA
-----------------
Body Soul & Spirit
Follows Jesus Christ with the Holy Spirit Helper John14:16 
#PostChemo #CancerFree Praise God Jesus #Conservative #UltraMAGA NoVax #pureblood
-----------------
#ProLife #NRA #RedWave #Trump2024 #TruePatriot #UltraMAGA #2A #freeSpeech
-----------------
AMERICANDREAM 🇺🇸 TRUMP ~ WHITE HATS~PATRIOT  #UltraMAGA @DonaldJTrumpJr @ElonMusk @DanScavino @realDonaldTrump @AmericanDream09 @GenFlynn @KariLake
-----------------
#AmericaFirst 🇺🇸 #UltraMAGA 🇺🇸 #HealthyEating💥 #RomanCath

### 14th iteration (republican)

In [268]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
# 5th: 'pro-biden', 'pro biden', 'pro-obama', 'pro obama', 'pro-harris', 'pro harris', 'pro-pelosi', 
#      'pro pelosi', 'pro-blinken', 'pro blinken', 'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
#      'pro-democrats', 'pro-democrat', 'pro democrats', 'pro democrat'
# 6th: 'anti-gop', 'anti gop'
# 7th: 'vote blue'
# 8th: 'voteblue', 'vote-blue'
# 9th: none filtered ('#joebiden', '#biden')
# 10th: 'vote ... 💙💙', '💙💙 ... vote', 'democrat ... 💙💙', '💙💙 ... democrat'
# 11th: 'proud democrat', 'lifelong democrat'
# 12th: 'blue wave', 'blue-wave', 'bluewave'
# 13th: 'ultramaga'

In [293]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [294]:
regex = alternative_regex([
    'trump2024'
])

In [295]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 16.26 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
444,1386124064578809857,Santiago Evies אביעז 🇮🇱,EviesSantiago,Global,"Writer, Researcher, Entrepreneur, Anti-communi...",False,1269,4989,4,1645,5641,2021-04-25 01:05:23+00:00
452,2854910420,D-Mo,DannyMo04,"New York, USA",@dannymo04 on Truth\n\n #Trump2024 #DemocratsA...,False,2209,3758,17,137487,38471,2014-10-13 23:17:55+00:00
1377,1399696921556566016,STOCKMASTER2000,STOCKMASTERJAY,A PRONOUN FREE SAFE SPACE ..,#UNCLEJAY #STOCKMASTER2000 #RAGEDAD #TRUMP2024...,False,2054,342,25,1841,28268,2021-06-01 11:59:08+00:00
2843,881548537116467203,Beya - Tunisia 🇹🇳,BeyaTunis,"El Marsa, Tunisia","Tunisian 🇹🇳 Vegan 🌱 Forensic Accountant, Maste...",False,274,439,3,7837,6443,2017-07-02 16:22:04+00:00
4562,894884057649172480,🇺🇸Kimberly Reanna🇺🇸,KimberlyReanna2,"Tennessee, USA",We the People are fed up🖕🏼Pure🩸Trump2024🇺🇸❌PLE...,False,3756,4962,1,2848,5894,2017-08-08 11:32:40+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
3668,1472330996938911751,T2ARVR,M69020846,,la France au français\n#Zemmour2022\n#trump202...,False,21,136,0,413,782,2021-12-18 22:20:51+00:00
10251,1594042426854756352,Sam Valk,valkyrienews17,"Next to a Homeless Camp, CA",American Nationalist | Online Safety Expert | ...,False,99,324,0,17896,7774,2022-11-19 18:58:33+00:00
1060,1615310597083602944,Hoppero🇺🇸,Hoppero01MAGA,"Wisconsin, USA",22 y/old Male-Straight-Conservative Republican...,False,994,1282,0,11889,3208,2023-01-17 11:31:06+00:00
1673,20994675,KatefromTexas,KatefromTexas,"Texas, USA",#MAGA #KAG #Deplorable #Trump2024 Proud Capita...,False,604,1259,2,46235,5061,2009-02-16 15:38:27+00:00


In [297]:
republican_userids.update(sdf['id'])

In [296]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

Writer, Researcher, Entrepreneur, Anti-communist. Author of: Words of the Soul, The Startup Jewish, Patriot 
🔯 I'm Jewish. #Trump2024
-----------------
@dannymo04 on Truth

 #Trump2024 #DemocratsAreDestroyingAmerica
#maga
-----------------
#UNCLEJAY #STOCKMASTER2000 #RAGEDAD #TRUMP2024 ALLCAPS #ALLTHETIME #2028 COVID & #TRUMP DERANGEMENT SYNDROMES ARE REAL! COMEBACKS ARE STRONGER THAN SETBACKS…
-----------------
Tunisian 🇹🇳 Vegan 🌱 Forensic Accountant, Masters in Public Administration. Support President Kaïs Saied Make Tunisia Great Again. A conservative 4 Trump2024
-----------------
We the People are fed up🖕🏼Pure🩸Trump2024🇺🇸❌PLEASE NO DM❌ #SaveOurChildren FUCK JOE BIDEN✌🏼
-----------------
America Media is 85% #FakeNews
#Impeach Criminal Joe Biden

#Trump2024
President Trump is the ONLY candidate I'll vote for in '24
TRUMP WON! 

#1A #2A #Life 👶
-----------------
Anti gauchiasse, anti woke, anti Sardine Ruisseau et toute la clique NUPES 😝😝 !!  #Trump2024 😇😇 #Zemmour2027 👍 #ZelenskyWar

### 15th iteration (republican)

In [299]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
# 5th: 'pro-biden', 'pro biden', 'pro-obama', 'pro obama', 'pro-harris', 'pro harris', 'pro-pelosi', 
#      'pro pelosi', 'pro-blinken', 'pro blinken', 'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
#      'pro-democrats', 'pro-democrat', 'pro democrats', 'pro democrat'
# 6th: 'anti-gop', 'anti gop'
# 7th: 'vote blue'
# 8th: 'voteblue', 'vote-blue'
# 9th: none filtered ('#joebiden', '#biden')
# 10th: 'vote ... 💙💙', '💙💙 ... vote', 'democrat ... 💙💙', '💙💙 ... democrat'
# 11th: 'proud democrat', 'lifelong democrat'
# 12th: 'blue wave', 'blue-wave', 'bluewave'
# 13th: 'ultramaga'
# 14th: 'trump2024'

In [301]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [309]:
regex = alternative_regex([
    'trump 2024', 'trump-2024'
])

In [310]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 16.20 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
10693,744743429234515968,Daily News,TopDailyTrendin,,Trump 2024,False,1477,11,1,557,124,2016-06-20 04:07:25+00:00
4248,1345239482925244416,LadyAnnihilate,lady_annihilate,"Dallas, TX",TRUMP 2024 TRUMP 2024 TRUMP 2024 TRUMP 2024 FU...,False,815,2221,2,22559,44595,2021-01-02 05:24:34+00:00
7564,747802933,Mystara,Mystara99,United States,"Trump 2024\n#Individualist #Constitutionalist,...",False,3525,3291,30,25092,38293,2012-08-09 19:32:08+00:00
11780,1583089165008265216,Merchant of Demonrats 🇺🇸🏴‍☠️,Mr_Maga2022,,"nuked, redo 10.0 MAGA !🇺🇲 BIKES, Dogs and Am...",False,243,357,0,1624,6208,2022-10-20 13:34:22+00:00
3746,1577404395318173696,No Sheep,slayprayslay,United States,✝️🇺🇸💁‍♀️ Native Californian turned Refugee. No...,False,959,2103,0,13723,3136,2022-10-04 21:05:27+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
9273,1503527036689784838,🍊MISFITPATRIOTS🍊,MISFIT_PATRIOTS,,🇺🇸LOVE MY COUNTRY 🇺🇸MAGA🇺🇸 FREE SPEECH 🇺🇸 NRA ...,False,7187,5777,8,16385,16889,2022-03-15 00:23:30+00:00
9531,561589946,deme,deme138,,💥 M🇺🇸A 🇺🇸G🇺🇸A💥 Trump 2024💥💪🏻🇺🇸,False,4522,4936,31,45347,52078,2012-04-24 00:33:10+00:00
9537,1519024161611464706,Robert Keene,RobertK72059194,Florida,Trump 2024...MAGA...,False,3726,4888,1,17328,10538,2022-04-26 18:43:05+00:00
788,1586482636049944576,Sling Shot,deanteamscene,,Christian NOT WOKE but Awake NOW. Independent ...,False,2056,2911,0,22854,5802,2022-10-29 22:18:49+00:00


In [312]:
republican_userids.update(sdf['id'])

In [311]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

Trump 2024
-----------------
TRUMP 2024 TRUMP 2024 TRUMP 2024 TRUMP 2024 FUCK YOUR FEELINGS!!!  The walking dead Joe Biden is a TREASONOUS SEDITIOUS ANTI-AMERICA POS! 🐝
-----------------
Trump 2024
#Individualist #Constitutionalist,
Jesus is the Son of God
-----------------
nuked,  redo 10.0  MAGA !🇺🇲 BIKES, Dogs and America , Trump 2024👌🇺🇲
 NY SUCKS 🖕Demonrats
-----------------
✝️🇺🇸💁‍♀️ Native Californian turned Refugee. Now live in a Red State. Trump 2024.
-----------------
Trump 2024
-----------------
🇺🇸 Trump 2024 🇺🇸 Company Owner 🇺🇸 Pure Blood 🇺🇸 Heavy Metal 🇺🇸 Christian 🇺🇸 Alligators, Cats, Rabbits 🇺🇸Time Travel🇺🇸 Proud Parent 🇺🇸 Married
-----------------
60 + male Vet married
Trump 2024.  To all you women athletes I support your fight to keep biological males out of women's sports.
-----------------
Conservative Christian🙏Adore my grandson Logan♥️Trump 2024🇺🇸Rebuild the Wall🇺🇸 IFBP🇺🇲Support Israel 🙏 GA🍑Blocked by Steve Schmidt+ ProLife
-----------------
Love God and Country. Pro

### 16th iteration (republican)

In [315]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
# 5th: 'pro-biden', 'pro biden', 'pro-obama', 'pro obama', 'pro-harris', 'pro harris', 'pro-pelosi', 
#      'pro pelosi', 'pro-blinken', 'pro blinken', 'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
#      'pro-democrats', 'pro-democrat', 'pro democrats', 'pro democrat'
# 6th: 'anti-gop', 'anti gop'
# 7th: 'vote blue'
# 8th: 'voteblue', 'vote-blue'
# 9th: none filtered ('#joebiden', '#biden')
# 10th: 'vote ... 💙💙', '💙💙 ... vote', 'democrat ... 💙💙', '💙💙 ... democrat'
# 11th: 'proud democrat', 'lifelong democrat'
# 12th: 'blue wave', 'blue-wave', 'bluewave'
# 13th: 'ultramaga'
# 14th: 'trump2024'
# 15th: 'trump 2024', 'trump-2024'

In [316]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [361]:
regex = alternative_regex([
    'trump2020', 'trump 2020', 'trump-2020',
    'trump2016', 'trump 2016', 'trump-2016',
    'santis2024', 'santis 2024', 'santis-2024'
])

In [362]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 68.81 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
4915,1492255807513272323,Gator1,Gator158590985,"Florida, USA","U.Fla. Law, RT/Like Not=Endorsements @NAUIWorl...",False,104,391,0,10123,4188,2022-02-11 21:54:57+00:00
7129,21600375,Manic Contrarian 🇺🇸 🇮🇱,ManicContrarian,"Santa Barbara, CA","Father, husband, observer, entrepreneur, econ ...",False,1893,3042,27,2980,9822,2009-02-22 21:29:24+00:00
4343,1565019503968362498,Brandon Morris,Brandon34737270,"Bangor, ME",Desantis 2024!!! American Patriot!!! Fuck Joe ...,False,123,2090,0,268,45,2022-08-31 16:51:58+00:00
4439,1519034404269608967,The Puertorican Conservative,Pr_C0nservative,,"#Enterpreneur, #Capitalist and #Desantis2024 o...",False,1323,693,4,2206,2160,2022-04-26 19:23:52+00:00
10343,395401013,Boi’d 🦅,Boi_dBoi_d,"Treasure Island, FL","Climate Realist, vivid reader of scientific pa...",False,581,910,19,3578,24280,2011-10-21 16:12:24+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
3285,93229339,Conrad Bezrad,ConradCortez,,Until they become conscious they'll never rebe...,False,1783,1091,25,1176,2804,2009-11-28 18:02:17+00:00
8662,3282717458,Tuckerwoman,Tuckerwoman2,"South Carolina, USA","Married, BS, RRT, NPS. Please follow. I follow...",False,1293,3218,0,9826,10026,2015-07-17 20:10:25+00:00
9388,826990247196110850,Dom,whyfreedom1975,"New York, NY",#Deplorable Father #CONSERVATIVE values belie...,False,7848,7637,3,44925,34689,2017-02-02 03:06:54+00:00
14086,373422852,Fascist Russian Asset 🔫,SamanthaPfaff,San Francisco East Bay,Serving up snark every day. DeSantis 2024 #1A ...,False,1837,3336,15,50576,53450,2011-09-14 15:11:46+00:00


In [364]:
republican_userids.update(sdf['id'])

In [363]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

U.Fla. Law, RT/Like Not=Endorsements @NAUIWorldwide #DeSantis2024 #BlueLivesMatter #GatorsBB @GatorsBB LitiGator, Investigator #Taekwondo Dep. Black Belt
-----------------
Father, husband, observer, entrepreneur, econ empowerment advocate, bleeding-heart-conservative, recovering politician... #DeSantis2024
-----------------
Desantis 2024!!! American Patriot!!! Fuck Joe Biden!!! I love the Bruins and The Broncos and 76ers and the Phillies
-----------------
#Enterpreneur, #Capitalist and #Desantis2024 or #Desantis2028 #Conservative. Commentator, political aficionado and a proud #American #ProLife #Traditionalist
-----------------
Climate Realist, vivid reader of scientific papers, political conservative, family person #TrumpdeSantis2024 #gladCO2isup #clintel English/French/Dutch
-----------------
Common sense Jewish conservative. Into the arts, health, family, pets. Haagen-Dazs no Ben & Jerry. I’m a woman not a pronoun. DeSantis 2024. Pro #Elon! 🇺🇸
-----------------
#MAGA #KAG #WeThePeop

One false positive here, but all the other are ok ;)

### 17th iteration (republican)

In [366]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
# 5th: 'pro-biden', 'pro biden', 'pro-obama', 'pro obama', 'pro-harris', 'pro harris', 'pro-pelosi', 
#      'pro pelosi', 'pro-blinken', 'pro blinken', 'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
#      'pro-democrats', 'pro-democrat', 'pro democrats', 'pro democrat'
# 6th: 'anti-gop', 'anti gop'
# 7th: 'vote blue'
# 8th: 'voteblue', 'vote-blue'
# 9th: none filtered ('#joebiden', '#biden')
# 10th: 'vote ... 💙💙', '💙💙 ... vote', 'democrat ... 💙💙', '💙💙 ... democrat'
# 11th: 'proud democrat', 'lifelong democrat'
# 12th: 'blue wave', 'blue-wave', 'bluewave'
# 13th: 'ultramaga'
# 14th: 'trump2024'
# 15th: 'trump 2024', 'trump-2024'
# 16th: 'trump 2020', 'trump 2016', 'santis-2024' ...

In [19]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [20]:
regex = alternative_regex([
    'americafirst', 'america first', 'america-first'
])

In [21]:
regex = r'(?:americafirst|america first|america-first).*(?:maga|trump|christian|constitution|god)|(?:maga|trump|christian|constitution|god).*(?:americafirst|america first|america-first)'

In [22]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 35.31 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
1996,1199771785148993536,Heerak Christian Kim for US Congress (Virginia-8),Heerak4Congress,"Arlington, Virginia",Heerak Christian Kim is running for US Congres...,True,787,4840,12,20,52920,2019-11-27 19:27:47+00:00
2973,25114781,T For Love and Country 🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸,ToriDi,USA,"Working mother, love of our Country, God, fair...",False,1416,1461,5,33669,75000,2009-03-18 17:41:23+00:00
3050,1462373693074591748,Dawn 🇺🇸,godsgreenearthy,,"America First! 🇺🇸 Christian Constitutionalist,...",False,2382,3899,0,19301,2882,2021-11-21 10:54:23+00:00
5050,1393667125978419200,Crispy1too,ChrisBlock2,"South West, Washington",Velvet hammer spreading #Truth … We need to ge...,False,501,591,2,501,658,2021-05-15 20:39:10+00:00
2076,1271867277600714752,Freedom Queen,FreedomQueen16,United States,♥️ God ✝️ USA🇺🇸 Constitutionalist 🇺🇸 Real Hous...,False,1534,2275,0,36317,6377,2020-06-13 18:10:06+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
10485,1595963532331327488,KarenAnn,realkarenfitz,,"Christian Conservative, America First 🇺🇸 woman...",False,370,1009,2,9694,3276,2022-11-25 02:12:28+00:00
12100,396265009,Deadwood Dick,depbosco,,Trump Conservative. America First. Retired Dep...,False,1603,2054,8,13914,10088,2011-10-23 00:53:33+00:00
12279,771032854113361920,Vivacious Valkyrie🇺🇸🇺🇸,LilMinxx2u,United States,"All American USAF Veteran 🇺🇸🇺🇸, Pro Law Enforc...",False,4119,5001,1,336722,14360,2016-08-31 17:12:12+00:00
2035,797791978280853504,Michele Nordan,michele_nordan,"Pompano Beach, FL","#AmericaFirst, ICU Trauma/Covid RN, Animal Lov...",False,263,819,0,42490,10788,2016-11-13 13:23:24+00:00


In [24]:
republican_userids.update(sdf['id'])

In [23]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

Heerak Christian Kim is running for US Congress for Virginia 8th District (Arlington, Alexandria, Falls Church, Fairfax) as MAGA America First candidate in 2024
-----------------
Working mother, love of our Country, God, fair opinions and #PresidentTrump #MAGA #AmericaFirst #SaveAmerica #CreaturesGreatandSmall
-----------------
America First! 🇺🇸 Christian Constitutionalist, Married to my soulmate, Lover of God & ALL of His Creation.
          ❤️🤍💙
-----------------
Velvet hammer spreading #Truth … We need to get to work fixing this BS! #God #Family #USA #MAGA #1A #2A #PureBloods #PATRIOT #NavyVet #AmericaFirst
-----------------
♥️ God ✝️ USA🇺🇸 Constitutionalist 🇺🇸 Real Housewife of Silicon Valley🍸 #MAGA 🇺🇸 #DJT45 🇺🇸 #AmericaFirst🇺🇸
-----------------
#AmericaFirst #1A #2A #USA #MAGA #Trump 
Resist/Libs R blocked
Dems/Libs/Leftist are destroying the USA 🤬👎
-----------------
Red America First is a grassroots org. dedicated to America 1st policies, Christian Faith (Red-Blood of Christ) & r

### Checkpoint 3

In [26]:
checkpoint(republican_userids, democrat_userids)

### 18th iteration (republican)

In [27]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
# 5th: 'pro-biden', 'pro biden', 'pro-obama', 'pro obama', 'pro-harris', 'pro harris', 'pro-pelosi', 
#      'pro pelosi', 'pro-blinken', 'pro blinken', 'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
#      'pro-democrats', 'pro-democrat', 'pro democrats', 'pro democrat'
# 6th: 'anti-gop', 'anti gop'
# 7th: 'vote blue'
# 8th: 'voteblue', 'vote-blue'
# 9th: none filtered ('#joebiden', '#biden')
# 10th: 'vote ... 💙💙', '💙💙 ... vote', 'democrat ... 💙💙', '💙💙 ... democrat'
# 11th: 'proud democrat', 'lifelong democrat'
# 12th: 'blue wave', 'blue-wave', 'bluewave'
# 13th: 'ultramaga'
# 14th: 'trump2024'
# 15th: 'trump 2024', 'trump-2024'
# 16th: 'trump 2020', 'trump 2016', 'santis-2024' ...
# 17th: 'america first' with different republican-oriented words (maga, trump, christian, constitution, god)

In [28]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [34]:
regex = alternative_regex([
    'trumpwon', 'trump won', 'trump-won'
])

In [35]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 16.19 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
1192,1477071867219304449,♱🇺🇸𝕄𝔸𝔾𝔸 𝑵𝒂𝒏𝒄𝒚 ن,45LVNancy,,Giving it another go 🥂 𝕃𝔼𝔾𝔸𝕃 𝕀𝕞𝕞𝕚𝕘𝕣𝕒𝕟𝕥 🇺🇸 Demo...,False,28030,3639,25,8784,163957,2022-01-01 00:20:21+00:00
1355,1835020626,BIKERS for America 🇺🇸🏍🇺🇸,Bikers4America,United States of America,#God✝ #Country 🇺🇸 #Bikers 🏍 #1stResponders 🚒🚓🚑...,False,14288,14566,64,109097,134895,2013-09-08 21:54:07+00:00
4009,701463939582918656,mega king 🇺🇸🇮🇱,megaking14,USA,THE TRUTH ~IT’S THERE ~FIND IT COUNTDOWN TO ET...,False,3334,3408,1,22757,27785,2016-02-21 17:50:10+00:00
1720,925100552257785857,Sabine Neumeyer,SabinevomBus,germanytrucker1@gmail.com,2015/2020 Parteilose OB Kandidatin & unabhängi...,False,1344,1179,16,9106,4854,2017-10-30 20:42:13+00:00
5395,1572291668795338752,Baby Jane 🙈🙉🙊,janesaddictionz,Your moms,Jane's just a girl who's tired of your sh*t!👅 ...,False,190,514,0,4301,5249,2022-09-20 18:28:58+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
10670,1589670337628749826,Double R,DoubleR81401,Deep in the ❤️ of Texas,"NO DMs PLEASE! I WON’T RESPOND! Awakened,🇺🇸 Am...",False,156,269,0,4219,2338,2022-11-07 17:27:12+00:00
12645,1589329991166316544,Ricky Ensor,RickyEnsor,Western North Carolina,"retired industrial maintenance, 63C10, tracked...",False,944,1849,1,51761,36038,2022-11-06 18:53:32+00:00
14015,1586719422491893760,KatJ ✝🇺🇸,momvogt1,USA ✝️🇺🇸,"God, Family, Country! \nWithout our Great✝️God...",False,653,779,0,14583,12937,2022-10-30 14:00:26+00:00
3606,32985717,CFeike,cfeike,"Virginia, USA","#MAGA, #KeepAmericaGreat, #TrumpWon “One day m...",False,1638,5023,2,45122,31817,2009-04-18 19:35:39+00:00


In [36]:
republican_userids.update(sdf['id'])

In [31]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

Giving it another go 🥂 𝕃𝔼𝔾𝔸𝕃 𝕀𝕞𝕞𝕚𝕘𝕣𝕒𝕟𝕥 🇺🇸 Democrats and RINOS are ᗪᗴᔕ丅ᖇᗝƳᎥᑎǤ ᗩᗰᗴᖇᎥᑕᗩ ⚤ #TrumpWon #FJB #2AshallNOTbeInfringed
-----------------
#God✝ #Country 🇺🇸 #Bikers 🏍 #1stResponders 🚒🚓🚑 #Military #Veterans 🎖 #Bikers4America ~ Original #2MBTDC 🇺🇸 #TrumpWon 😎 🚫NO PORN or DMs 🚫 #BLOCKED
-----------------
THE TRUTH ~IT’S THERE ~FIND IT COUNTDOWN TO ETERNITY 
✝️👨‍👩‍👧‍👦🇺🇸⚓️  #MAGA #KAG #TrumpWon     🚫bitcoin
-----------------
2015/2020 Parteilose OB Kandidatin & unabhängige Unternehmerin/TrumpWon/#MAGA/#USA🇺🇸#rumble:NewsMeyer https://t.co/2gfwpOEwhz
-----------------
Jane's just a girl who's tired of your sh*t!👅 
N🚫 poison in my blood🚫💉🩸
PROUD TO BE AMERICAN! GOD BLESS 🇺🇸 🙌
BIDEN = SMALL D* ENERGY🤏
Trump Won! 🗳
-----------------
(I/Am)🇨🇦 !!! #adoptdontshop 🐾 anti-corruption #Trudeaufortreason #TrumpWon The Truth never minds being questioned.
-----------------
#TruthSeeker #Antiwar #USConstitution #NewsJunkie #INFJ  #TrumpWon #GodTheCreator I'm here, I'm there, I'm everywhere.
-----------

There were a few description of accounts outside of US, but they still seem to represent the same political views.

### 19th iteration (republican)

In [38]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
# 5th: 'pro-biden', 'pro biden', 'pro-obama', 'pro obama', 'pro-harris', 'pro harris', 'pro-pelosi', 
#      'pro pelosi', 'pro-blinken', 'pro blinken', 'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
#      'pro-democrats', 'pro-democrat', 'pro democrats', 'pro democrat'
# 6th: 'anti-gop', 'anti gop'
# 7th: 'vote blue'
# 8th: 'voteblue', 'vote-blue'
# 9th: none filtered ('#joebiden', '#biden')
# 10th: 'vote ... 💙💙', '💙💙 ... vote', 'democrat ... 💙💙', '💙💙 ... democrat'
# 11th: 'proud democrat', 'lifelong democrat'
# 12th: 'blue wave', 'blue-wave', 'bluewave'
# 13th: 'ultramaga'
# 14th: 'trump2024'
# 15th: 'trump 2024', 'trump-2024'
# 16th: 'trump 2020', 'trump 2016', 'santis-2024' ...
# 17th: 'america first' with different republican-oriented words (maga, trump, christian, constitution, god)
# 18th: 'trump won' and so on..

In [39]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [48]:
regex = r'ultra.{,18}maga'

In [49]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 16.21 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
9069,1313159237946806278,ultra AMC 🍊MAGA dragonfly2034 🪐,dragonfly2034,,AMC Ultra 🍊MAGA Company 🪐\n#AMC 🍿 $APE 🚀\n\n...,False,1776,2803,1,57592,34284,2020-10-05 16:49:42+00:00
171,1392069711685378048,🇺🇸NuclearMAGA Terry🍊🇺🇸,MC_Mad_Man3,Motor City,"Third account 😩, ULTRA #MAGA FOREVER #BackTheB...",False,16859,14701,1,67639,76340,2021-05-11 10:51:17+00:00
8528,114572713,Egor Titov,ethome01,"Missouri, USA",Love my family and the United States of Americ...,False,19809,17834,5,28383,24023,2010-02-15 21:45:44+00:00
14069,1587727960047583232,BrandonObiden,BrandonObiden,"Maine, USA",Escapee from New York. Recovering attorney. H...,False,1276,2995,1,7380,6372,2022-11-02 08:47:19+00:00
2847,1619222308832280577,John Lutz,TrumpInaugMarch,"Harrisonburg, VA",ClubRetro/JohnLutz Approx 273rdacctsince'12! N...,False,969,2344,1,10028,17828,2023-01-28 06:34:31+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
13587,1441609902766493697,Maphoo,K8MGV,Michigan,Ultra-Mega-MAGA Conservative who believes the ...,False,165,393,0,3853,1096,2021-09-25 03:46:21+00:00
13208,1163870244676390912,Bea Goode,BeaGoode3,The once Golden State,There's no history of anything until it happen...,False,2508,5000,2,82360,9141,2019-08-20 17:48:01+00:00
4066,1520792110600966144,chaotic world led by fools,conservicanUSA,,American Citizen\nULTRA MEGA MAGA !\nProud NY ...,False,309,548,0,12482,7901,2022-05-01 15:48:00+00:00
9527,1621749714,USMC3048,drabyUSMC,"Texas, USA",ULTRA #MAGA #KAG #USMC #TrumpTrain #2A 🇺🇸❤️🇺🇸❤...,False,2263,3061,2,42633,18974,2013-07-26 02:09:39+00:00


In [51]:
republican_userids.update(sdf['id'])

In [50]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

AMC Ultra 🍊MAGA Company  🪐
#AMC 🍿  $APE 🚀

triggered replies always= transvestite groomers. avowed wokaphobic
-----------------
Third account 😩, ULTRA #MAGA FOREVER #BackTheBlue!! #BackTheMilitary IFBP MAGA #CCOT #TCOT #1A #2A *- I BLOCK STUPID -* TruthSocial - MC_MAD_MAN
-----------------
Love my family and the United States of America. No DM's Make America Great Again 🇺🇸ULTRA #MAGA #KAG #TRUMP🇺🇸 #STLCARDS
-----------------
Escapee from New York.  Recovering attorney. Hiding in a bunker where it's so cold no one will come looking for me. ULTRA MEGA MAGA. #DefundNPR
-----------------
ClubRetro/JohnLutz Approx 273rdacctsince'12! NEVEREVERGIVEUP! #JESUSISGOD!!! #ULTRAGODMAGA! #WeGotThis! #GodWins! TrumpCURRENTPotus/JFK,Jr.CURRENTVP! QQQ! #2023
-----------------
Patriot 🇺🇸, Pure Blood #45Won Deplorably Ultra #MAGAForever #2AMatters Baseball Fan Trump/Won terry_trumper on TRUTH
-----------------
🌹Carmelite Sec🌹Jesus❤️Catholic, Divine Mercy, Rosary🌹#Ultra #MAGA #PATRIOT; DAR; Mayflower; FSA

4 false positives, not that much, so adding :)

### 20th iteration (republican)

In [54]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
# 5th: 'pro-biden', 'pro biden', 'pro-obama', 'pro obama', 'pro-harris', 'pro harris', 'pro-pelosi', 
#      'pro pelosi', 'pro-blinken', 'pro blinken', 'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
#      'pro-democrats', 'pro-democrat', 'pro democrats', 'pro democrat'
# 6th: 'anti-gop', 'anti gop'
# 7th: 'vote blue'
# 8th: 'voteblue', 'vote-blue'
# 9th: none filtered ('#joebiden', '#biden')
# 10th: 'vote ... 💙💙', '💙💙 ... vote', 'democrat ... 💙💙', '💙💙 ... democrat'
# 11th: 'proud democrat', 'lifelong democrat'
# 12th: 'blue wave', 'blue-wave', 'bluewave'
# 13th: 'ultramaga'
# 14th: 'trump2024'
# 15th: 'trump 2024', 'trump-2024'
# 16th: 'trump 2020', 'trump 2016', 'santis-2024' ...
# 17th: 'america first' with different republican-oriented words (maga, trump, christian, constitution, god)
# 18th: 'trump won' and so on..
# 19th: 'ultra ... maga'

In [55]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [71]:
regex = r'(?:proud|lifelong|forever|go)\b.{,5}republican'

In [72]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 26.59 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
10697,1292107633147752451,OldDwB,OldDwB,"Las Vegas, NV",A home for angry satire.\nLifelong Republican....,False,3,1,0,103,1325,2020-08-08 14:37:41+00:00
8033,910327496,Valerie Montana,BigSky3450,,A lifelong Republican who no longer recognizes...,False,469,412,7,19045,33746,2012-10-28 13:41:08+00:00
9437,866746746839355392,David Olson,daveolson4142,"Aurora, CO","I am a proud Republican, living in a region su...",False,93,10,0,26960,4910,2017-05-22 20:05:02+00:00
5759,1585763035016097792,Brian Ference,BrianFerence1,Arizona,"Lifelong Republican, Critical Thinker, Freedom...",False,329,401,2,5055,5137,2022-10-27 22:39:13+00:00
1640,816624635303362560,On My Mind,OnMyMind54,,"Proud Republican, Entrepreneur, Small Business...",False,575,942,26,8321,15111,2017-01-04 12:37:40+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
3772,914803616,Laura Bogues (monroe),LauraMonroe318,Louisiana,I'm A Simple Person Who Hides A Thousand Feeli...,False,242,293,1,7202,858,2012-10-30 14:48:34+00:00
4156,1452909312884441091,Mike Hunt,GregJesus7,"Las Vegas, NV",I am a proud Christian Fuck Biden What you are...,False,6,41,0,13,88,2021-10-26 08:05:57+00:00
7119,824340595434872832,Pete F 🇺🇦,petef46,Undisclosed.,Lifelong Republican. Never-Trumper and general...,False,147,161,1,7894,15016,2017-01-25 19:38:08+00:00
1717,39284255,Jennifer Hardy,jenparkerhardy,,"Former lifelong Republican, now recovering fro...",False,158,439,5,7692,2787,2009-05-11 16:31:20+00:00


In [51]:
# republican_userids.update(sdf['id'])

In [73]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

A home for angry satire.
Lifelong Republican.  Never a Nazi! I recommend against following me.
-----------------
A lifelong Republican who no longer recognizes the GOP. In 2022 I’ll be voting DEM down the ballot.
-----------------
I am a proud Republican, living in a region surrounded by wokes. They believe we are not allowed an opinion. They want to take away our freedom by all means.
-----------------
Lifelong Republican, Critical Thinker, Freedom Lover, My opinions are my own

Joined May 2022

https://t.co/QsMBMdYn0V
-----------------
Proud Republican, Entrepreneur, Small Business Owner, #Technologist, #Innovator, #Parrothead #FinsUp #MAGA #MASA #MAGANOMICS Full support of @POTUS #Buckeyes
-----------------
A lifelong Republican who voted Blue to save this country. A Texan who grew up in the Midwest.b
-----------------
32 county Irish man. love sport tv and  politics. Proud Republican 🇮🇪
-----------------
Elon Musk stan  - minnesota sports diehard- NASCAR- Morgan Wallen stan  PROUD 

Very interesting! Those are real republicans, who mostly do not associate themselves with Trump! Leaving this, as it may be interesting to look at these later.

### 21th iteration (republican)

In [75]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
# 5th: 'pro-biden', 'pro biden', 'pro-obama', 'pro obama', 'pro-harris', 'pro harris', 'pro-pelosi', 
#      'pro pelosi', 'pro-blinken', 'pro blinken', 'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
#      'pro-democrats', 'pro-democrat', 'pro democrats', 'pro democrat'
# 6th: 'anti-gop', 'anti gop'
# 7th: 'vote blue'
# 8th: 'voteblue', 'vote-blue'
# 9th: none filtered ('#joebiden', '#biden')
# 10th: 'vote ... 💙💙', '💙💙 ... vote', 'democrat ... 💙💙', '💙💙 ... democrat'
# 11th: 'proud democrat', 'lifelong democrat'
# 12th: 'blue wave', 'blue-wave', 'bluewave'
# 13th: 'ultramaga'
# 14th: 'trump2024'
# 15th: 'trump 2024', 'trump-2024'
# 16th: 'trump 2020', 'trump 2016', 'santis-2024' ...
# 17th: 'america first' with different republican-oriented words (maga, trump, christian, constitution, god)
# 18th: 'trump won' and so on..
# 19th: 'ultra ... maga'
# 20th: none filtered ('proud ... republican' and so on)

In [76]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [77]:
regex = alternative_regex([
    'maga2024', 'maga-2024', 'maga 2024',
    'maga2020', 'maga-2020', 'maga 2020',
    'maga2016', 'maga-2016', 'maga 2016'
])

In [78]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 16.24 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
11208,2693483342,Red Barnett,yehezkell,Living in the Material World,Conservative. Dedicated Zionist. Most ppl R ot...,False,2311,2390,19,6889,29541,2014-07-30 17:29:16+00:00
5876,800175372067213312,💪🇺🇸Jeff Palmer🇺🇸💪,kjeffpalmer,,#JESUSCHRIST 4ourkids #Trumpbestpresident #Vet...,False,16107,16368,3,5062,92411,2016-11-20 03:14:10+00:00
6554,815960928663572486,John Shook MAGA,sivan1040,"Georgia, USA",Baptist Minister MAGA 2020 Build the Wall. Fol...,False,4470,4389,3,197830,227415,2017-01-02 16:40:20+00:00
11159,2882837082,mikes,MSalazar1950,United States,"Freedom loving, America loving, Support the US...",False,1225,1559,1,147999,21088,2014-10-30 03:56:27+00:00
2450,468930205,JennyJen,JennyJen68,"Broward County, FL",#MiamiNative Live & work in #BrowardCounty #Fl...,False,3943,3807,9,17008,21520,2012-01-20 02:04:56+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
12076,303845475,Vasilichenko,IamDariusDaKING,Section 8,"Mild (sometimes Hard) Conservative, Entreprene...",False,132,320,0,3369,2081,2011-05-23 14:26:59+00:00
14072,1109987470035152896,JinxieB🇺🇸,BJinxie,"Arizona, USA","HAPPILY MARRIED, conservative, wife, mom and A...",False,6873,4587,0,7311,13032,2019-03-25 01:16:50+00:00
2516,333604264,Barbara S Mayer,BarbaraSMayer,Indiana,Conservative Christian. Proud Republican from ...,False,1069,3208,26,37285,21794,2011-07-11 20:03:15+00:00
9343,845681715066613761,🇺🇸🍔 UltraMAGADoubleBaconCheezyBorger©®™ 🍔🇺🇸,CheezyBorger,"Indiana, USA",Hot n juicy Cheezyborger 🍔🍟Can't forget NO LIS...,False,5212,5579,1,321852,275909,2017-03-25 17:00:07+00:00


In [80]:
republican_userids.update(sdf['id'])

In [79]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

Conservative. Dedicated Zionist. Most ppl R other ppl. Their thoughts R someone else's opinions. Their lives a mimicry. #MAGA2024 Self-appointed Islamophobe
-----------------
#JESUSCHRIST 4ourkids #Trumpbestpresident #Vets #USAFIRST #2A  #NRA#Rockymountains #DraintheSwamp #2000Mules #MAGA🇺🇸🇺🇸🙏#MAGA2024 #LGB
💪🇺🇲🇺🇲💪
-----------------
Baptist Minister MAGA 2020 Build the Wall. Followed by Gen Flynn
-----------------
Freedom loving, America loving, Support the US Constitution. Drain the Swamp. MAGA 2024!!
-----------------
#MiamiNative Live & work in #BrowardCounty #Florida 🏝. happily married 32 years ❤️#MAGA2024 🇺🇸  
#IStandWithTrump 💯
-----------------
#MAGA2022 #MAGA2024
-----------------
A JFK Democrat until the Reagan years. Today's Democrat party is what the party of JFK was trying to protect us against. #MAGA2024 🇺🇸
-----------------
I love my country and absolutely HATE woke garbage!! MAGA 2024!!
-----------------
Don't tread on me, männlich gelesen, he/him, Maga2024, 🗽💛🚿, 🥝, 🇩🇪,🇮🇱

### 22th iteration (republican)

In [82]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
# 5th: 'pro-biden', 'pro biden', 'pro-obama', 'pro obama', 'pro-harris', 'pro harris', 'pro-pelosi', 
#      'pro pelosi', 'pro-blinken', 'pro blinken', 'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
#      'pro-democrats', 'pro-democrat', 'pro democrats', 'pro democrat'
# 6th: 'anti-gop', 'anti gop'
# 7th: 'vote blue'
# 8th: 'voteblue', 'vote-blue'
# 9th: none filtered ('#joebiden', '#biden')
# 10th: 'vote ... 💙💙', '💙💙 ... vote', 'democrat ... 💙💙', '💙💙 ... democrat'
# 11th: 'proud democrat', 'lifelong democrat'
# 12th: 'blue wave', 'blue-wave', 'bluewave'
# 13th: 'ultramaga'
# 14th: 'trump2024'
# 15th: 'trump 2024', 'trump-2024'
# 16th: 'trump 2020', 'trump 2016', 'santis-2024' ...
# 17th: 'america first' with different republican-oriented words (maga, trump, christian, constitution, god)
# 18th: 'trump won' and so on..
# 19th: 'ultra ... maga'
# 20th: none filtered ('proud ... republican' and so on)
# 21th: 'maga2024', 'maga2020', 'maga2016', 'maga 2024', ...

In [83]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [90]:
regex = r'(?:god|maga|conservative|patriot|christian|pro-?life|constitution)\b.*trump supporter.*(?:god|maga|conservative|patriot|christian|pro-?life|constitution)'

In [91]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 34.93 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
3671,1412610705438646277,RealQueenMaatGirl,RealMaat,New Orleans Louisiana,"Patriotic freedom fighter. MAGA girl, conserva...",False,120,211,0,1160,2447,2021-07-07 03:13:43+00:00
3031,769610225972305920,Faf,Realfafman,"New York, NY","Patriot, Trump Supporter (same thing) Lover of...",False,2300,3806,2,10688,21474,2016-08-27 18:59:11+00:00
7752,1585954553668698112,Red Pill Ike,ike61927455,,"Patriot, Previously known as Eddy English tryi...",False,157,252,0,115,519,2022-10-28 11:20:21+00:00
7232,247748988,Snake Doctor,SnakeDoctorC,,"NAVY VET, S.E.R.E. Whitby Island, NRA, Loves...",False,334,695,5,264,4593,2011-02-05 13:47:28+00:00
5500,2478580825,"Deplorable, Neanderthal Chris White",ChrisWh11261958,"KS, United States","#MAGA #KAGA, Eagle Scout, Fiscal Conservative,...",False,15793,15993,14,364507,158056,2014-05-05 16:21:19+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
1650,1459155457201909763,MAGA Mike Doyle,mike_Spuddy,Hell.,Patriot!!! Happily married 2A TRUMP supporter....,False,552,1081,0,10028,4646,2021-11-12 13:45:59+00:00
1616,598825150,Connie Roberts,Pitbullconnie,"Dallas, Texas","Proud conservative, physician, animal lover. ...",False,1137,1135,20,5448,9364,2012-06-04 01:32:59+00:00
7445,26870656,Susie,susiem57,"Pennsylvania, USA","Catholic Conservative Trump supporter, Wife an...",False,7514,7379,3,121057,110683,2009-03-26 22:24:27+00:00
9014,1976638741,"🇺🇸 ❌ULTRA Mega MAGA, Alpha Male, Tim 🍊❌🇺🇸",PersonalTims,FL,Retired Army 25 years. Proud Patriot! Love to ...,False,2347,3466,1,3008,31106,2013-10-20 22:23:10+00:00


In [92]:
republican_userids.update(sdf['id'])

In [89]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

Patriotic freedom fighter. MAGA girl, conservative Trump supporter.  live on YouTube every Monday, Wed & Friday 6pm EST. #MAGA #Patriot #ProLife #Conservatism
-----------------
Patriot, Trump Supporter (same thing) Lover of Family, Country, Freedom, and Life. #MAGA. We are in trouble, but it's not too late. Don't vote for idiots.
-----------------
Patriot, Previously known as Eddy English trying to rebuild my profile ,Massive Trump supporter , Maga
I search for truth and spread it far and wide
-----------------
NAVY VET,  S.E.R.E. Whitby Island,  NRA, Loves Freedom, Cancer Survivor, Father, Grandfather, #MAGA #Trump Supporter  100% Conservative
-----------------
#MAGA #KAGA, Eagle Scout, Fiscal Conservative, son of WWII vet, father and grandfather, Proud Trump Supporter! 1A 2A #MAGA #KAG IFB All Patriots! NO DMs!!
-----------------
Love Family, Country, God, Proud Trump supporter, Veterans Rock! #2AStrong, Stand for our beautiful 🇺🇸🇺🇸, #ProLife, 🧩 Let’s go Brandon, Truckers Rock 🇺🇸
---

### 23th iteration (republican)

In [82]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
# 5th: 'pro-biden', 'pro biden', 'pro-obama', 'pro obama', 'pro-harris', 'pro harris', 'pro-pelosi', 
#      'pro pelosi', 'pro-blinken', 'pro blinken', 'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
#      'pro-democrats', 'pro-democrat', 'pro democrats', 'pro democrat'
# 6th: 'anti-gop', 'anti gop'
# 7th: 'vote blue'
# 8th: 'voteblue', 'vote-blue'
# 9th: none filtered ('#joebiden', '#biden')
# 10th: 'vote ... 💙💙', '💙💙 ... vote', 'democrat ... 💙💙', '💙💙 ... democrat'
# 11th: 'proud democrat', 'lifelong democrat'
# 12th: 'blue wave', 'blue-wave', 'bluewave'
# 13th: 'ultramaga'
# 14th: 'trump2024'
# 15th: 'trump 2024', 'trump-2024'
# 16th: 'trump 2020', 'trump 2016', 'santis-2024' ...
# 17th: 'america first' with different republican-oriented words (maga, trump, christian, constitution, god)
# 18th: 'trump won' and so on..
# 19th: 'ultra ... maga'
# 20th: none filtered ('proud ... republican' and so on)
# 21th: 'maga2024', 'maga2020', 'maga2016', 'maga 2024', ...
# 22th: 'trump supporter' and other republican-oriented words -//-

In [93]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [84]:
regex = alternative_regex([
    'trump supporter'
])

In [114]:
subregex = r'(?:god|maga|conservative|patriot|christian|pro-?life|constitution|trump|family|mother|father|wife)\b'
subregex2 = r'(?:god|maga|conservative|patriot|christian|pro-?life|constitution|trump)\b'
subregex3 = r'(?:maga|trump)'
regex = r'.*'.join([subregex] * 2 + [subregex2] + [subregex3])

In [115]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

[########################################] | 100% Completed | 66.97 s


,id,name,username,location,description,verified,followers,followings,listed,tweets_liked,tweets,created_at
8487,3102583924,NancyM,mamanan77,"Chicago, IL",Wife and Mother. Conservative. Unflinching Cub...,False,265,1040,1,614,1385,2015-03-21 19:15:31+00:00
1943,187613807,wvf,wvsteelville,"Dallas, TX","Lover of God, Family, Freedom, Liberty Small G...",False,1100,3837,2,12335,14367,2010-09-06 18:18:52+00:00
7282,467909441,clarena roux ♛,clarenaroux,Cantabria-Spain,"Anticomunista, defensora de los presos polític...",False,1439,1886,33,22835,76940,2012-01-18 23:56:36+00:00
8419,429183327,#MAGA Info 🇺🇸✝️,KovacsNick,The World Wide Web,Jesus is my Savior & Trump is my president! In...,False,639,1799,58,5704,77209,2011-12-05 17:22:20+00:00
9229,1146346244274397184,Michaela,Michaela6718903,USA,"Constitution loving, conservative, pro-life, C...",False,4311,5110,0,31215,7835,2019-07-03 09:13:37+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
6203,772891219059961856,just b,BeRadEddy,"Happy Valley, OR",Gender Solid deplorable. Critical Thinker. Cli...,False,1052,1481,1,38891,22949,2016-09-05 20:16:41+00:00
6234,1519396638594322438,Maurice,MauriceNK2022,Kuwait 🇰🇼,AB Canadian 🇨🇦 Conservative Patriot Married to...,False,809,995,0,11481,14668,2022-04-27 19:28:53+00:00
8861,823555652832219137,GSD Puppies,GSDPuppies1,"Ohio, USA","German Shepherd & legal gun owner, conservativ...",False,1382,3213,11,55639,56042,2017-01-23 15:39:03+00:00
9077,838569536,Bill E. Bomber,bill_e_bomber,"Somewhere, USA",Conservative Christian Patriot. President Dona...,False,2012,1832,3,57177,9319,2012-09-21 21:08:34+00:00


In [117]:
republican_userids.update(sdf['id'])

In [116]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')

Wife and Mother. Conservative. Unflinching Cubs and Blackhawk Fan! God, Family, Country. #Constitution #MAGA
-----------------
Lover of God, Family, Freedom, Liberty Small Govt, Pro-Religion, Pro-Life, Pro-Free Markets, Pro-2A, Pro-Constitution, ProTrump & #IAmTheNRA #2A #1A #MAGA #KAGA
-----------------
Anticomunista, defensora de los presos políticos, de la Democracia, detesto el socialismo, soy #Patriot y apoyo incondicionalmente a #Trump #MAGA #VivaTrump
-----------------
Jesus is my Savior & Trump is my president! Info to #MAGA in all aspects of life, with tweets to educate & entertain. God bless America 🖖 #TrumpIsTheBestPOTUS
-----------------
Constitution loving, conservative, pro-life, Christian. 🇺🇸 Mom to USMC vet.🇺🇸 America 1st. 1A/2A. 🚫porn, crypto, or DMs. MAGA!
-----------------
God first! American patriot. US Constitution. SNARKY! Trump is my President! Retired LEO! BUY AMERICAN 🇺🇸 My dogs ❤️❤️ 🍄 #RollTideRoll
-----------------
Christian-Husband(52y)-Father-Retired Pharma

Only 2 false positives, seems to be acceptable!

### Checkpoint 4

In [119]:
len(republican_userids), len(democrat_userids)

(11374, 13612)

In [120]:
checkpoint(republican_userids, democrat_userids)

### 24h iteration (??)

In [121]:
# previous ones
# 1st: 'anti-trump', 'anti-maga', 'anti-carlson', 'anti-foxnews', 'anti-republican', 'fuck trump'
# 2nd: none filtered ('anti-biden', 'anti-obama', 'anti-democrat', 'fuck biden', 'biden dementia',
#                     'fuck democrat', 'fuck blm', 'anti-blm', 'anti democrat', 'anti-left', 'fuck left')
# 3rd: 'ultra maga', 'ultra-maga'
# 4th: 'pro-trump', 'pro-maga', 'pro trump', 'pro maga', 'pro-republican', 'pro republican'
# 5th: 'pro-biden', 'pro biden', 'pro-obama', 'pro obama', 'pro-harris', 'pro harris', 'pro-pelosi', 
#      'pro pelosi', 'pro-blinken', 'pro blinken', 'pro clinton', 'pro-clinton', 'pro-nuland', 'pro nuland'
#      'pro-democrats', 'pro-democrat', 'pro democrats', 'pro democrat'
# 6th: 'anti-gop', 'anti gop'
# 7th: 'vote blue'
# 8th: 'voteblue', 'vote-blue'
# 9th: none filtered ('#joebiden', '#biden')
# 10th: 'vote ... 💙💙', '💙💙 ... vote', 'democrat ... 💙💙', '💙💙 ... democrat'
# 11th: 'proud democrat', 'lifelong democrat'
# 12th: 'blue wave', 'blue-wave', 'bluewave'
# 13th: 'ultramaga'
# 14th: 'trump2024'
# 15th: 'trump 2024', 'trump-2024'
# 16th: 'trump 2020', 'trump 2016', 'santis-2024' ...
# 17th: 'america first' with different republican-oriented words (maga, trump, christian, constitution, god)
# 18th: 'trump won' and so on..
# 19th: 'ultra ... maga'
# 20th: none filtered ('proud ... republican' and so on)
# 21th: 'maga2024', 'maga2020', 'maga2016', 'maga 2024', ...
# 22th: 'trump supporter' and other republican-oriented words -//-
# 23th: combinations of republican-oriented words

In [122]:
fdf = df[~df['id'].isin(democrat_userids | republican_userids)]

In [123]:
regex = alternative_regex([
    'TODO'
])

In [ ]:
with ProgressBar(dt=2.0):
    sdf = description_search(fdf, regex).compute()
sdf

In [117]:
# ??.update(sdf['id'])

In [ ]:
for row in list(sdf.itertuples())[:100]:
    print(row.description)
    print('-----------------')